# Import libraries

In [1]:
import pypots
import os
import sys
from pypots.nn.functional import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012 as preprocess_physionet2012
from missingData.toolkits import toolkits
import math
import scipy.stats as st

2025-04-28 21:08:45.276543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745885325.292356  354834 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745885325.297332  354834 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 21:08:45.313572: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidge


████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



# Loading database

### Standard Scaler 

In [2]:
set_random_seed()
physionet2012_dataset_standard = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)
print(physionet2012_dataset_standard.keys())

2025-04-28 21:08:53 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-04-28 21:08:53 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-04-28 21:08:53 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-04-28 21:08:53 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-04-28 21:08:53 [INFO]: Loaded successfully!
2025-04-28 21:09:07 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-04-28 21:09:07 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-04-28 21:09:08 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [3]:
scaler_standard = physionet2012_dataset_standard["scaler"]

In [4]:
dataset_for_training_standard, dataset_for_validating_standard, dataset_for_testing_ori_standard, dataset_for_testing_standard = toolkits.separating_dataset(physionet2012_dataset_standard)

# Creating indicating mask and get test_X_ori

### Standard Scaler

In [5]:
dataset_for_testing_ori_standard_norm = toolkits.dict_to_list(dataset_for_testing_ori_standard)
dataset_for_testing_standard_norm = toolkits.dict_to_list(dataset_for_testing_standard)
indicating_mask_variable_standard_norm, test_X_ori_variable_standard_norm = toolkits.components_mae(dataset_for_testing_ori_standard_norm, dataset_for_testing_standard_norm)
indicating_mask_variable_standard_norm = toolkits.pre_reshape(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.pre_reshape(test_X_ori_variable_standard_norm)
indicating_mask_variable_standard_norm = toolkits.reshape_variable(indicating_mask_variable_standard_norm)
test_X_ori_variable_standard_norm = toolkits.reshape_variable(test_X_ori_variable_standard_norm)

# Inicialize the models

### SAITS

<h4>Inicialize new model</h4>

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 21:54:46 [INFO]: No given device, using default device: cuda
2025-04-22 21:54:46 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250422_T215446
2025-04-22 21:54:46 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250422_T215446/tensorboard
2025-04-22 21:54:46 [INFO]: Using customized MAE as the training loss function.
2025-04-22 21:54:46 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 21:54:46 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


<h4> Inicialize model</h4>

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-28 21:10:27 [INFO]: No given device, using default device: cuda
2025-04-28 21:10:27 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-04-28 21:10:27 [INFO]: Using customized MAE as the training loss function.
2025-04-28 21:10:27 [INFO]: Using customized MSE as the validation metric function.


2025-04-28 21:10:27 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


### BRITS

<h4>Inicialize new model</h4>

In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: Using customized MAE as the training loss function.
2025-04-22 18:16:43 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 18:16:43 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


<h4> Inicialize model</h4>

In [7]:
brits = BRITS(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-27 22:06:01 [INFO]: No given device, using default device: cuda
2025-04-27 22:06:01 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-04-27 22:06:01 [INFO]: Using customized MAE as the training loss function.
2025-04-27 22:06:01 [INFO]: Using customized MSE as the validation metric function.
2025-04-27 22:06:01 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


### USGAN

<h4>Inicialize new model</h4>

In [8]:
us_gan = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


<h4> Inicialize model</h4>

In [8]:
us_gan = USGAN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-27 22:06:01 [INFO]: No given device, using default device: cuda
2025-04-27 22:06:01 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-04-27 22:06:01 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


### GPVAE

<h4>Inicialize new model</h4>

In [9]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


<h4> Inicialize model</h4>

In [9]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-27 22:06:01 [INFO]: No given device, using default device: cuda
2025-04-27 22:06:01 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-04-27 22:06:01 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


### MRNN

<h4>Inicialize new model</h4>

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-22 18:16:43 [INFO]: No given device, using default device: cuda
2025-04-22 18:16:43 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250422_T181643
2025-04-22 18:16:43 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250422_T181643/tensorboard
2025-04-22 18:16:43 [INFO]: Using customized RMSE as the training loss function.
2025-04-22 18:16:43 [INFO]: Using customized MSE as the validation metric function.
2025-04-22 18:16:43 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


<h4> Inicialize model</h4>

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset_standard['n_steps'],
    n_features=physionet2012_dataset_standard['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-04-27 22:06:04 [INFO]: No given device, using default device: cuda
2025-04-27 22:06:04 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-04-27 22:06:04 [INFO]: Using customized RMSE as the training loss function.
2025-04-27 22:06:04 [INFO]: Using customized MSE as the validation metric function.
2025-04-27 22:06:04 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


# Train/Load the models

### SAITS

<h4>Train</h4>

In [7]:
saits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 21:54:58 [INFO]: Epoch 001 - training loss (MAE): 0.7233, validation MSE: 6.7652
2025-04-22 21:55:03 [INFO]: Epoch 002 - training loss (MAE): 0.5383, validation MSE: 6.7408
2025-04-22 21:55:08 [INFO]: Epoch 003 - training loss (MAE): 0.4956, validation MSE: 6.7218
2025-04-22 21:55:12 [INFO]: Epoch 004 - training loss (MAE): 0.4643, validation MSE: 6.6987
2025-04-22 21:55:16 [INFO]: Epoch 005 - training loss (MAE): 0.4392, validation MSE: 6.6806
2025-04-22 21:55:21 [INFO]: Epoch 006 - training loss (MAE): 0.4207, validation MSE: 6.6686
2025-04-22 21:55:24 [INFO]: Epoch 007 - training loss (MAE): 0.4065, validation MSE: 6.6527
2025-04-22 21:55:28 [INFO]: Epoch 008 - training loss (MAE): 0.3953, validation MSE: 6.6548
2025-04-22 21:55:32 [INFO]: Epoch 009 - training loss (MAE): 0.3866, validation MSE: 6.6511
2025-04-22 21:55:36 [INFO]: Epoch 010 - training loss (MAE): 0.3798, validation MSE: 6.6584
2025-04-22 21:55:36 [INFO]: Finished training. The best model is from epoch#9.
2

<h4> Load</h4>

In [7]:
saits.load("tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots")

2025-04-28 21:11:57 [INFO]: Model loaded successfully from tutorial_results/imputation/saits/20250422_T181642/SAITS.pypots


### BRITS

<h4>Train</h4>

In [12]:
brits.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 18:20:14 [INFO]: Epoch 001 - training loss (MAE): 0.9371, validation MSE: 6.7840
2025-04-22 18:22:18 [INFO]: Epoch 002 - training loss (MAE): 0.7328, validation MSE: 6.7456
2025-04-22 18:24:23 [INFO]: Epoch 003 - training loss (MAE): 0.6822, validation MSE: 6.7326
2025-04-22 18:26:17 [INFO]: Epoch 004 - training loss (MAE): 0.6576, validation MSE: 6.7294
2025-04-22 18:28:36 [INFO]: Epoch 005 - training loss (MAE): 0.6427, validation MSE: 6.7280
2025-04-22 18:30:47 [INFO]: Epoch 006 - training loss (MAE): 0.6318, validation MSE: 6.7267
2025-04-22 18:32:52 [INFO]: Epoch 007 - training loss (MAE): 0.6229, validation MSE: 6.7294
2025-04-22 18:35:07 [INFO]: Epoch 008 - training loss (MAE): 0.6149, validation MSE: 6.7316
2025-04-22 18:37:26 [INFO]: Epoch 009 - training loss (MAE): 0.6087, validation MSE: 6.7332
2025-04-22 18:37:26 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-04-22 18:37:26 [INFO]: Finished training. The best model is from epoc

<h4>Load</h4>

In [12]:
brits.load("tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots")

2025-04-27 22:06:10 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/20250422_T181643/BRITS.pypots


### USGAN

<h4>Train</h4>

In [13]:
us_gan.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 18:41:35 [INFO]: Epoch 001 - generator training loss: 0.4346, discriminator training loss: 0.1907, validation loss: 6.7750
2025-04-22 18:45:20 [INFO]: Epoch 002 - generator training loss: 0.3701, discriminator training loss: 0.0566, validation loss: 6.7282
2025-04-22 18:49:07 [INFO]: Epoch 003 - generator training loss: 0.3347, discriminator training loss: 0.0374, validation loss: 6.7004
2025-04-22 18:52:56 [INFO]: Epoch 004 - generator training loss: 0.3165, discriminator training loss: 0.0311, validation loss: 6.6837
2025-04-22 18:56:43 [INFO]: Epoch 005 - generator training loss: 0.3020, discriminator training loss: 0.0281, validation loss: 6.6737
2025-04-22 19:00:32 [INFO]: Epoch 006 - generator training loss: 0.2923, discriminator training loss: 0.0265, validation loss: 6.6685
2025-04-22 19:04:19 [INFO]: Epoch 007 - generator training loss: 0.2877, discriminator training loss: 0.0252, validation loss: 6.6691
2025-04-22 19:08:07 [INFO]: Epoch 008 - generator training los

<h4>Load</h4>

In [13]:
us_gan.load("tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots")

2025-04-27 22:06:12 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/20250422_T181643/USGAN.pypots


### GPVAE

<h4>Train</h4>

In [14]:
gp_vae.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 19:15:55 [INFO]: Epoch 001 - training loss (default): 26027.5423, validation loss: 22882.8125
2025-04-22 19:16:01 [INFO]: Epoch 002 - training loss (default): 22877.1633, validation loss: 22807.7179
2025-04-22 19:16:07 [INFO]: Epoch 003 - training loss (default): 22842.6190, validation loss: 22788.8947
2025-04-22 19:16:14 [INFO]: Epoch 004 - training loss (default): 22832.1135, validation loss: 22783.6572
2025-04-22 19:16:20 [INFO]: Epoch 005 - training loss (default): 22827.5731, validation loss: 22780.0771
2025-04-22 19:16:27 [INFO]: Epoch 006 - training loss (default): 22821.9797, validation loss: 22773.8286
2025-04-22 19:16:34 [INFO]: Epoch 007 - training loss (default): 22814.3062, validation loss: 22767.3447
2025-04-22 19:16:40 [INFO]: Epoch 008 - training loss (default): 22810.7426, validation loss: 22771.6522
2025-04-22 19:16:46 [INFO]: Epoch 009 - training loss (default): 22808.6126, validation loss: 22762.6405
2025-04-22 19:16:52 [INFO]: Epoch 010 - training loss (

<h4>Load</h4>

In [14]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots")

2025-04-27 22:06:15 [INFO]: Model loaded successfully from tutorial_results/imputation/gp_vae/20250422_T181643/GPVAE.pypots


### MRNN

<h4>Train</h4>

In [15]:
mrnn.fit(train_set=dataset_for_training_standard, val_set=dataset_for_validating_standard)

2025-04-22 19:17:30 [INFO]: Epoch 001 - training loss (RMSE): 0.7504, validation MSE: 7.3129
2025-04-22 19:17:47 [INFO]: Epoch 002 - training loss (RMSE): 0.5341, validation MSE: 7.2684
2025-04-22 19:18:05 [INFO]: Epoch 003 - training loss (RMSE): 0.4959, validation MSE: 7.2556
2025-04-22 19:18:22 [INFO]: Epoch 004 - training loss (RMSE): 0.4612, validation MSE: 7.2489
2025-04-22 19:18:40 [INFO]: Epoch 005 - training loss (RMSE): 0.4541, validation MSE: 7.2464
2025-04-22 19:18:59 [INFO]: Epoch 006 - training loss (RMSE): 0.4473, validation MSE: 7.2449
2025-04-22 19:19:16 [INFO]: Epoch 007 - training loss (RMSE): 0.4298, validation MSE: 7.2458
2025-04-22 19:19:34 [INFO]: Epoch 008 - training loss (RMSE): 0.4340, validation MSE: 7.2485
2025-04-22 19:19:51 [INFO]: Epoch 009 - training loss (RMSE): 0.4229, validation MSE: 7.2492
2025-04-22 19:19:51 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-04-22 19:19:51 [INFO]: Finished training. The best model is 

<h4>Load</h4>

In [15]:
mrnn.load("tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots")

2025-04-27 22:06:19 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/20250422_T181643/MRNN.pypots


# Testing stage

### SAITS

#### Standard Scaler

In [10]:
saits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, saits)
saits_imputation_standard = toolkits.pre_reshape(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.desnormalization(saits_imputation_standard, scaler_standard)
saits_imputation_standard = toolkits.reshape_variable(saits_imputation_standard)
saits_imputation_standard_ori = toolkits.reshape_variable(saits_imputation_standard_ori)

### BRITS

#### Standard Scaler

In [17]:
brits_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, brits)
brits_imputation_standard = toolkits.pre_reshape(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.desnormalization(brits_imputation_standard, scaler_standard)
brits_imputation_standard = toolkits.reshape_variable(brits_imputation_standard)
brits_imputation_standard_ori = toolkits.reshape_variable(brits_imputation_standard_ori)

### USGAN

In [18]:
usgan_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, us_gan)
usgan_imputation_standard = toolkits.pre_reshape(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.desnormalization(usgan_imputation_standard, scaler_standard)
usgan_imputation_standard = toolkits.reshape_variable(usgan_imputation_standard)
usgan_imputation_standard_ori = toolkits.reshape_variable(usgan_imputation_standard_ori)

### GPVAE

In [19]:
gpvae_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, gp_vae)
gpvae_imputation_standard = toolkits.pre_reshape(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.desnormalization(gpvae_imputation_standard, scaler_standard)
gpvae_imputation_standard = toolkits.reshape_variable(gpvae_imputation_standard)
gpvae_imputation_standard_ori = toolkits.reshape_variable(gpvae_imputation_standard_ori)

### MRNN

In [20]:
mrnn_imputation_standard = toolkits.model_imputation(dataset_for_testing_standard, mrnn)
mrnn_imputation_standard = toolkits.pre_reshape(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.desnormalization(mrnn_imputation_standard, scaler_standard)
mrnn_imputation_standard = toolkits.reshape_variable(mrnn_imputation_standard)
mrnn_imputation_standard_ori = toolkits.reshape_variable(mrnn_imputation_standard_ori)

# Calculate mean absolute error

In [8]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

## SAITS

### Calculate MAE and AE

In [11]:
testing_mae_saits_variables_standard, testing_ae_saits_variables_standard = toolkits.calculate_mae(saits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for SAITS model

#### Difference between gender group

In [23]:
diff_saits_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 1, 2, variables)
diff_saits_gender_subgroup

[('Cholesterol', 0.5369467090194768),
 ('TroponinI', 0.21670823641985287),
 ('SaO2', 0.20063240934906212),
 ('Lactate', 0.10301878449328433),
 ('Urine', 0.06260750938123472)]

#### Difference between age group

In [24]:
diff_saits_age_subgroup = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 4, 5, variables)
diff_saits_age_subgroup

[('Cholesterol', 0.71459003341906),
 ('TroponinI', 0.5566432157859624),
 ('ALP', 0.35925495394620277),
 ('SaO2', 0.195697379474154),
 ('Bilirubin', 0.19557828928286905)]

#### Difference between ICUType group

In [25]:
diff_saits_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 7, variables)
diff_saits_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 8, variables)
diff_saits_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 6, 9, variables)
diff_saits_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 7, 8, variables)
diff_saits_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 7, 9, variables)
diff_saits_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 8, 9, variables)

In [26]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_saits_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_saits_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_saits_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_saits_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_saits_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_saits_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('TroponinI', 0.9325070965069859), ('Cholesterol', 0.9139836378262194), ('TroponinT', 0.6100616346293688), ('Albumin', 0.322760249031397), ('AST', 0.2527107910385458)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('TroponinI', 1.071422685673967), ('SaO2', 1.05884942799818), ('ALP', 0.7533389736410095), ('TroponinT', 0.5510962875933088), ('Albumin', 0.20918888387069867)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.6292154996436101), ('TroponinT', 0.563187305642908), ('Albumin', 0.24242616927070348), ('Bilirubin', 0.20322886146603605), ('Glucose', 0.1836342906325531)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.2740858480789576), ('Cholesterol', 0.8991669232297483), ('ALP', 0.8536263789449293), ('Temp', 0.2721045848889434), ('Glucose', 0.22842306840229232)]
----------------------------
Diferença(MAE) IC

In [27]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[31])
print(variables[6])
print(variables[0])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
TroponinI
Cholesterol
ALP
TroponinT


#### Difference between BMI group

In [28]:
diff_saits_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 11, variables)
diff_saits_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 12, variables)
diff_saits_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 13, variables)
diff_saits_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 14, variables)
diff_saits_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 15, variables)
diff_saits_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 10, 16, variables)

diff_saits_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 12, variables)
diff_saits_classification_low_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 13, variables)
diff_saits_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 14, variables)
diff_saits_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 15, variables)
diff_saits_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 11, 16, variables)

diff_saits_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 13, variables)
diff_saits_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 14, variables)
diff_saits_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 15, variables)
diff_saits_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 12, 16, variables)

diff_saits_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 14, variables)
diff_saits_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 15, variables)
diff_saits_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 13, 16, variables)


diff_saits_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 14, 15, variables)
diff_saits_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 14, 16, variables)

diff_saits_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_saits_variables_standard, 15, 16, variables)

In [29]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_saits_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_saits_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_saits_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_saits_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_saits_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_saits_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.0695107866797067), ('Bilirubin', 0.9915185253107142), ('TroponinI', 0.6146236391081507), ('TroponinT', 0.443330279599206), ('Glucose', 0.41663925906179167)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 0.289557588458789), ('SaO2', 0.2683085747984237), ('TroponinI', 0.16227929548420128), ('Bilirubin', 0.13978310423485213), ('PaCO2', 0.13696079385519522)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.41945464826892875), ('TroponinI', 0.2691987270297762), ('SaO2', 0.24053118518222205), ('TroponinT', 0.22793864497708735), ('Temp', 0.15260042386152797)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8884223528865745), ('SaO2', 0.25746552497276776), ('Albumin', 0.20186188486109413), ('Glucose', 0.1748223370

In [30]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_saits_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_saits_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_saits_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_saits_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_saits_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 0.9982778336968872), ('Bilirubin', 0.8517354210758621), ('Cholesterol', 0.602423448733534), ('Glucose', 0.4573483121011198), ('TroponinI', 0.4523443436239494)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.203609745779102), ('Bilirubin', 1.136682060340596), ('TroponinI', 0.8838223661379269), ('Cholesterol', 0.7323205085436738), ('Albumin', 0.45063840894416585)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.22901353915177), ('Bilirubin', 1.0050969115937654), ('Ph', 0.8831229167215342), ('Albumin', 0.5967438929186778), ('Glucose', 0.5914615961061891)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.272840141868911), ('Bilirubin', 1.1380893524477826), ('TroponinT', 0.7408628267792727), ('Albumin', 0.6101654410062806), ('Glucose', 0.39559457289

In [31]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_saits_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_saits_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_saits_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_saits_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('TroponinI', 0.4314780225139775), ('Bilirubin', 0.284946639264734), ('ALP', 0.2053319120822148), ('TroponinT', 0.1926569364019618), ('Cholesterol', 0.12989705981013977)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8949578072986595), ('ALP', 0.23073570545488276), ('Albumin', 0.187628752476664), ('Mg', 0.1764365766369922), ('Cholesterol', 0.16955196245074788)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.1489113978033532), ('Cholesterol', 0.8242085364526168), ('TroponinI', 0.4523443436239494), ('Bilirubin', 0.28635393137192056), ('ALP', 0.2745623081720239)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('TroponinT', 0.9380847511572676), ('Cholesterol', 0.8267873952728071), ('ALP', 0.23797859208480934), ('ALT', 0.2252208677412318), ('Biliru

In [32]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_saits_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_saits_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_saits_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Ph', 0.8907623616431242), ('TroponinI', 0.37049138280073846), ('Cholesterol', 0.29944902226088765), ('Glucose', 0.15595708586656248), ('Albumin', 0.14610548397451195)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinT', 0.9562544614013914), ('Cholesterol', 0.9541055962627566), ('TroponinI', 0.8838223661379269), ('Albumin', 0.15952703206211472), ('K', 0.11861308211336918)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 0.9566844550829469), ('TroponinT', 0.7454278147553058), ('TroponinI', 0.5011836548138614), ('Glucose', 0.17570699329093742), ('FiO2', 0.1685193157969586)]
----------------------------


In [33]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_saits_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_saits_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.1014464734027822), ('Ph', 0.8916481822575759), ('Cholesterol', 0.6546565740018689), ('TroponinI', 0.5133309833371884), ('Glucose', 0.19586702321444854)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Ph', 0.8969041137930027), ('TroponinT', 0.8906198267566967), ('Cholesterol', 0.6572354328220592), ('Glucose', 0.3316640791574999), ('ALT', 0.18581185150181573)]
----------------------------


In [34]:

print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_saits_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('TroponinI', 0.38263871132406546), ('TroponinT', 0.21082664664608552), ('K', 0.1659808046890613), ('Glucose', 0.13579705594305136), ('HCO3', 0.13126580142306704)]
----------------------------


In [35]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[32])
print(variables[0])
print(variables[6])
print(variables[31])
print(variables[5])

Top 5 variáveis BMI
-----------------------
TroponinT
ALP
Cholesterol
TroponinI
Bilirubin


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [36]:
indices = [6, 31, 28, 16, 33]

results_bootstrap_saits_female = []

for i in indices:
       results_bootstrap_saits_female.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [37]:
means_bootstraps_saits_female,standards_deviations_saits_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_female)

print(means_bootstraps_saits_female)
print(standards_deviations_saits_female)


[np.float64(1.373748843926726), np.float64(0.37275858362109265), np.float64(0.5524716886956614), np.float64(0.3326085230412998), np.float64(0.31286199594986686)]
[np.float64(0.2763156656324914), np.float64(0.07217211517611831), np.float64(0.13870822329993635), np.float64(0.02584678967416644), np.float64(0.008810708363842035)]


#### Calculating lower bound and upper bound

In [38]:
lower_bounds_saits_female, upper_bounds_saits_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_female, means_bootstraps_saits_female,standards_deviations_saits_female)

print(lower_bounds_saits_female)
print(upper_bounds_saits_female)

[np.float64(0.832180090922831), np.float64(0.2313038371878241), np.float64(0.28060856666824663), np.float64(0.2819497461639518), np.float64(0.29559332487845064)]
[np.float64(1.915317596930621), np.float64(0.5142133300543612), np.float64(0.8243348107230762), np.float64(0.3832672999186478), np.float64(0.3301306670212831)]


In [39]:
lower_bounds_saits_female, upper_bounds_saits_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_female)

print(lower_bounds_saits_female)
print(upper_bounds_saits_female)

[np.float64(0.7956493962302685), np.float64(0.23914515302182113), np.float64(0.3143323717583187), np.float64(0.28345713374277015), np.float64(0.29614027976911866)]
[np.float64(1.8935062048888518), np.float64(0.5251975154710748), np.float64(0.8523403898000738), np.float64(0.38416654520917304), np.float64(0.3303962235070448)]


#### Mean values of lower bound and upper bound


In [40]:
mean_values_ci_saits_female = toolkits.calc_mean_values_ci(lower_bounds_saits_female, upper_bounds_saits_female)

print(mean_values_ci_saits_female)

[np.float64(1.34457780055956), np.float64(0.382171334246448), np.float64(0.5833363807791963), np.float64(0.3338118394759716), np.float64(0.3132682516380817)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [41]:
indices = [6, 31, 28, 16, 33]

results_bootstrap_saits_male = []

for i in indices:
       results_bootstrap_saits_male.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [42]:
means_bootstraps_saits_male,standards_deviations_saits_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_male)

print(means_bootstraps_saits_male)
print(standards_deviations_saits_male)


[np.float64(0.8454456157021425), np.float64(0.5870766871262704), np.float64(0.35223551572908746), np.float64(0.43548421708030166), np.float64(0.37531705704424695)]
[np.float64(0.143840246031478), np.float64(0.17114481857085853), np.float64(0.03324243128664382), np.float64(0.03333202574969339), np.float64(0.011765254887986593)]


#### Calculating lower bound and upper bound

In [43]:
lower_bounds_saits_male, upper_bounds_saits_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_male, means_bootstraps_saits_male,standards_deviations_saits_male)

print(lower_bounds_saits_male)
print(upper_bounds_saits_male)

[np.float64(0.5635239139530652), np.float64(0.2516390065867458), np.float64(0.28708154764871807), np.float64(0.3701546470791409), np.float64(0.3522575811948594)]
[np.float64(1.1273673174512198), np.float64(0.922514367665795), np.float64(0.41738948380945684), np.float64(0.5008137870814624), np.float64(0.3983765328936345)]


In [44]:
lower_bounds_saits_male, upper_bounds_saits_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_male)

print(lower_bounds_saits_male)
print(upper_bounds_saits_male)

[np.float64(0.5523831343602811), np.float64(0.2433625140681217), np.float64(0.29165645089184855), np.float64(0.3738390338339372), np.float64(0.3531061209160324)]
[np.float64(1.120114725634226), np.float64(0.9316484704602163), np.float64(0.422275351458329), np.float64(0.5046641338457889), np.float64(0.3993208596160408)]


#### Mean values of lower bound and upper bound


In [45]:
mean_values_ci_saits_male = toolkits.calc_mean_values_ci(lower_bounds_saits_male, upper_bounds_saits_male)

print(mean_values_ci_saits_male)

[np.float64(0.8362489299972535), np.float64(0.5875054922641689), np.float64(0.35696590117508875), np.float64(0.43925158383986307), np.float64(0.37621349026603657)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [46]:
indices = [6, 32, 0, 28, 5]

results_bootstrap_saits_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_saits_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [47]:
means_bootstraps_saits_more_than_or_equal_to_65_subgroup,standards_deviations_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_saits_more_than_or_equal_to_65_subgroup)
print(standards_deviations_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.683311639982926), np.float64(0.30663355208076215), np.float64(0.5370040452635078), np.float64(0.3591155379611528), np.float64(0.21794633697042903)]
[np.float64(0.17629860778493864), np.float64(0.06688429215540845), np.float64(0.11717502776507074), np.float64(0.03234980702293529), np.float64(0.024597338668250326)]


#### Calculating lower bound and upper bound


In [48]:
lower_bounds_saits_more_than_or_equal_to_65_subgroup, upper_bounds_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_more_than_or_equal_to_65_subgroup, means_bootstraps_saits_more_than_or_equal_to_65_subgroup,standards_deviations_saits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_saits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_saits_more_than_or_equal_to_65_subgroup)


[np.float64(0.33777271819989346), np.float64(0.17554274832470673), np.float64(0.30734521095648826), np.float64(0.29571108128937873), np.float64(0.16973643906512398)]
[np.float64(1.0288505617659585), np.float64(0.43772435583681757), np.float64(0.7666628795705273), np.float64(0.4225199946329269), np.float64(0.2661562348757341)]


In [49]:
lower_bounds_saits_more_than_or_equal_to_65_subgroup, upper_bounds_saits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_saits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.35597846481394957), np.float64(0.1921553038861846), np.float64(0.3316215241760741), np.float64(0.30078791770110647), np.float64(0.17282016195369798)]
[np.float64(1.05250639691288), np.float64(0.45107730908058646), np.float64(0.7907962758262245), np.float64(0.4277171466443936), np.float64(0.2679791242553792)]


#### Mean values of lower bound and upper bound

In [50]:
mean_values_ci_saits_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_more_than_or_equal_to_65_subgroup , upper_bounds_saits_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_saits_more_than_or_equal_to_65_subgroup)

[np.float64(0.7042424308634148), np.float64(0.3216163064833855), np.float64(0.5612089000011493), np.float64(0.36425253217275), np.float64(0.22039964310453858)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [12]:
indices = [6, 32, 0, 28, 5]

results_bootstrap_saits_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_saits_less_than_65_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [52]:
means_bootstraps_saits_less_than_65_subgroup,standards_deviations_saits_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_less_than_65_subgroup)

print(means_bootstraps_saits_less_than_65_subgroup)
print(standards_deviations_saits_less_than_65_subgroup)

[np.float64(1.208289487828978), np.float64(0.27428476133353286), np.float64(0.8925227730258256), np.float64(0.5564066607647613), np.float64(0.4129615508935271)]
[np.float64(0.797583775582912), np.float64(0.08184517674601052), np.float64(0.37591954421336243), np.float64(0.13554168671539277), np.float64(0.07186205317743216)]


#### Calculating lower bound and upper bound

In [53]:
lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_less_than_65_subgroup, means_bootstraps_saits_less_than_65_subgroup, standards_deviations_saits_less_than_65_subgroup)

print(lower_bounds_saits_less_than_65_subgroup)
print(upper_bounds_saits_less_than_65_subgroup)


[np.float64(-0.3549459869670064), np.float64(0.11387116260303709), np.float64(0.1557340052829228), np.float64(0.2907498363987804), np.float64(0.2721145148106579)]
[np.float64(2.7715249626249623), np.float64(0.4346983600640286), np.float64(1.6293115407687284), np.float64(0.8220634851307422), np.float64(0.5538085869763962)]


In [13]:
lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_less_than_65_subgroup)

print(lower_bounds_saits_less_than_65_subgroup)
print(upper_bounds_saits_less_than_65_subgroup)


[np.float64(0.0), np.float64(0.1355628153311714), np.float64(0.32717498098171865), np.float64(0.3236336315051599), np.float64(0.28589890622038067)]
[np.float64(2.29797867480385), np.float64(0.45557198397096177), np.float64(1.707836159771366), np.float64(0.8528012189301762), np.float64(0.5622029093938989)]


#### Mean values of lower bound and upper bound

In [55]:
mean_values_ci_saits_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_less_than_65_subgroup, upper_bounds_saits_less_than_65_subgroup)

print(mean_values_ci_saits_less_than_65_subgroup)

[np.float64(1.148989337401925), np.float64(0.29742976054709863), np.float64(1.0301826786945896), np.float64(0.5934558922523314), np.float64(0.4252249762823711)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [56]:
indices = [28, 31, 6, 0, 32]

results_bootstrap_saits_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_1_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [57]:
means_bootstraps_saits_ICUType_1_subgroup, standards_deviations_saits_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_1_subgroup)

print(means_bootstraps_saits_ICUType_1_subgroup)
print(standards_deviations_saits_ICUType_1_subgroup)


[np.float64(0.5175232711207629), np.float64(1.4278963717988322), np.float64(0.9129259916787572), np.float64(0.21492256770089987), np.float64(0.7220408300156814)]
[np.float64(0.18227494979941367), np.float64(0.4755777669967603), np.float64(0.1910354968552305), np.float64(0.03940171351431769), np.float64(0.1529220761683557)]


#### Calculating lower bound and upper bound

In [58]:
lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_1_subgroup, means_bootstraps_saits_ICUType_1_subgroup, standards_deviations_saits_ICUType_1_subgroup)

print(lower_bounds_saits_ICUType_1_subgroup)
print(upper_bounds_saits_ICUType_1_subgroup)

[np.float64(0.16027093423006578), np.float64(0.4957810766372005), np.float64(0.5385032980737907), np.float64(0.13769662828367207), np.float64(0.42231906828461335)]
[np.float64(0.87477560801146), np.float64(2.360011666960464), np.float64(1.2873486852837237), np.float64(0.2921485071181277), np.float64(1.0217625917467494)]


In [59]:
lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_1_subgroup)

print(lower_bounds_saits_ICUType_1_subgroup)
print(upper_bounds_saits_ICUType_1_subgroup)

[np.float64(0.2721307111818871), np.float64(0.543638988066526), np.float64(0.5363954702369058), np.float64(0.141052404752942), np.float64(0.44118206996901976)]
[np.float64(0.9469395469158568), np.float64(2.427082702067996), np.float64(1.3002812933748946), np.float64(0.2972861634721159), np.float64(1.0472229543060139)]


#### Mean values of lower bound and upper bound

In [60]:
mean_values_ci_saits_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_1_subgroup, upper_bounds_saits_ICUType_1_subgroup)

print(mean_values_ci_saits_ICUType_1_subgroup)

[np.float64(0.609535129048872), np.float64(1.485360845067261), np.float64(0.9183383818059002), np.float64(0.21916928411252895), np.float64(0.7442025121375169)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [61]:
indices = [28, 31, 6, 0, 32]

results_bootstrap_saits_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_2_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [62]:
means_bootstraps_saits_ICUType_2_subgroup, standards_deviations_saits_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_2_subgroup)

print(means_bootstraps_saits_ICUType_2_subgroup)
print(standards_deviations_saits_ICUType_2_subgroup)


[np.float64(0.302330846940043), np.float64(0.4805505000645951), np.float64(0.0), np.float64(0.11446282246482933), np.float64(0.11144690325298823)]
[np.float64(0.022484822368596478), np.float64(0.14194048224116887), np.float64(0.0), np.float64(0.015170346694063874), np.float64(0.011097118126151561)]


#### Calculating lower bound and upper bound

In [63]:
lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_2_subgroup, means_bootstraps_saits_ICUType_2_subgroup, standards_deviations_saits_ICUType_2_subgroup)

print(lower_bounds_saits_ICUType_2_subgroup)
print(upper_bounds_saits_ICUType_2_subgroup)

[np.float64(0.2582614048988133), np.float64(0.202352266923657), np.float64(0.0), np.float64(0.08472948931147786), np.float64(0.08969695139354456)]
[np.float64(0.3464002889812727), np.float64(0.7587487332055332), np.float64(0.0), np.float64(0.1441961556181808), np.float64(0.1331968551124319)]


In [64]:
lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_2_subgroup)

print(lower_bounds_saits_ICUType_2_subgroup)
print(upper_bounds_saits_ICUType_2_subgroup)

[np.float64(0.26052604936651513), np.float64(0.18986505956715696), np.float64(0.0), np.float64(0.08483659567234057), np.float64(0.09044232359616046)]
[np.float64(0.34895492675589923), np.float64(0.7805805876880018), np.float64(0.0), np.float64(0.14509193298337147), np.float64(0.1344022433581575)]


#### Mean values of lower bound and upper bound

In [65]:
mean_values_ci_saits_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_2_subgroup, upper_bounds_saits_ICUType_2_subgroup)

print(mean_values_ci_saits_ICUType_2_subgroup)

[np.float64(0.3047404880612072), np.float64(0.48522282362757935), np.float64(0.0), np.float64(0.11496426432785603), np.float64(0.11242228347715899)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [66]:
indices = [28, 31, 6, 0, 32]

results_bootstrap_saits_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_3_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [67]:
means_bootstraps_saits_ICUType_3_subgroup, standards_deviations_saits_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_3_subgroup)

print(means_bootstraps_saits_ICUType_3_subgroup)
print(standards_deviations_saits_ICUType_3_subgroup)


[np.float64(1.5841230287885042), np.float64(0.3442581782489468), np.float64(0.8792872952354504), np.float64(0.9678921773734542), np.float64(0.17010714776707878)]
[np.float64(0.4580019823961283), np.float64(0.06377201884422147), np.float64(0.17910323571645184), np.float64(0.3821815015371616), np.float64(0.04131366246733019)]


#### Calculating lower bound and upper bound

In [68]:
lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_3_subgroup, means_bootstraps_saits_ICUType_3_subgroup, standards_deviations_saits_ICUType_3_subgroup)

print(lower_bounds_saits_ICUType_3_subgroup)
print(upper_bounds_saits_ICUType_3_subgroup)

[np.float64(0.6864556384441448), np.float64(0.21926731809286304), np.float64(0.5282514037166168), np.float64(0.2188301988031781), np.float64(0.08913385726166742)]
[np.float64(2.4817904191328637), np.float64(0.4692490384050305), np.float64(1.2303231867542839), np.float64(1.7169541559437302), np.float64(0.25108043827249016)]


In [69]:
lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_3_subgroup)

print(lower_bounds_saits_ICUType_3_subgroup)
print(upper_bounds_saits_ICUType_3_subgroup)

[np.float64(0.7721555830654729), np.float64(0.21874596646993222), np.float64(0.672715728591764), np.float64(0.3739342724649222), np.float64(0.10112949143394352)]
[np.float64(2.5686964869598774), np.float64(0.470684071940567), np.float64(1.2507690080254117), np.float64(1.822936301165187), np.float64(0.2598147214626444)]


#### Mean values of lower bound and upper bound

In [70]:
mean_values_ci_saits_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_3_subgroup, upper_bounds_saits_ICUType_3_subgroup)

print(mean_values_ci_saits_ICUType_3_subgroup)

[np.float64(1.6704260350126752), np.float64(0.3447150192052496), np.float64(0.9617423683085878), np.float64(1.0984352868150546), np.float64(0.18047210644829395)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [71]:
indices = [28, 31, 6, 0, 32]

results_bootstrap_saits_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_saits_ICUType_4_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [72]:
means_bootstraps_saits_ICUType_4_subgroup, standards_deviations_saits_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_ICUType_4_subgroup)

print(means_bootstraps_saits_ICUType_4_subgroup)
print(standards_deviations_saits_ICUType_4_subgroup)


[np.float64(0.4348812171138142), np.float64(0.7754464465352181), np.float64(0.7335534335513758), np.float64(0.3829863186704514), np.float64(0.15835507919870295)]
[np.float64(0.09256210857353321), np.float64(0.3532831689263103), np.float64(0.1823292780305236), np.float64(0.0683837223603779), np.float64(0.026023271121236684)]


#### Calculating lower bound and upper bound

In [73]:
lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_ICUType_4_subgroup, means_bootstraps_saits_ICUType_4_subgroup, standards_deviations_saits_ICUType_4_subgroup)

print(lower_bounds_saits_ICUType_4_subgroup)
print(upper_bounds_saits_ICUType_4_subgroup)

[np.float64(0.25346281797660297), np.float64(0.08302415909547001), np.float64(0.37619461528435943), np.float64(0.24895668571532434), np.float64(0.10735040504115778)]
[np.float64(0.6162996162510255), np.float64(1.4678687339749663), np.float64(1.0909122518183922), np.float64(0.5170159516255785), np.float64(0.20935975335624812)]


In [74]:
lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_ICUType_4_subgroup)

print(lower_bounds_saits_ICUType_4_subgroup)
print(upper_bounds_saits_ICUType_4_subgroup)

[np.float64(0.27939241666052433), np.float64(0.37283400758291424), np.float64(0.36301224178524266), np.float64(0.25870839304650867), np.float64(0.11097505309520649)]
[np.float64(0.6381956673551215), np.float64(1.8003804692113732), np.float64(1.0613737960735214), np.float64(0.5258771348855341), np.float64(0.21241468020378465)]


#### Mean values of lower bound and upper bound

In [75]:
mean_values_ci_saits_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_ICUType_4_subgroup, upper_bounds_saits_ICUType_4_subgroup)

print(mean_values_ci_saits_ICUType_4_subgroup)

[np.float64(0.4587940420078229), np.float64(1.0866072383971437), np.float64(0.712193018929382), np.float64(0.3922927639660214), np.float64(0.16169486664949556)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [76]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_undefined_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [77]:
means_bootstraps_saits_undefined_classification_subgroup, standards_deviations_saits_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_undefined_classification_subgroup)

print(means_bootstraps_saits_undefined_classification_subgroup)
print(standards_deviations_saits_undefined_classification_subgroup)


[np.float64(0.22202724599185142), np.float64(0.3634481036858109), np.float64(0.5361061308000131), np.float64(0.6154688951137669), np.float64(0.35255617546895457)]
[np.float64(0.045046997511669136), np.float64(0.05301608593423815), np.float64(0.16599527717279897), np.float64(0.20926874088448058), np.float64(0.10138902903226112)]


#### Calculating lower bound and upper bound

In [78]:
lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_undefined_classification_subgroup, means_bootstraps_saits_undefined_classification_subgroup, standards_deviations_saits_undefined_classification_subgroup)

print(lower_bounds_saits_undefined_classification_subgroup)
print(upper_bounds_saits_undefined_classification_subgroup)

[np.float64(0.1337367532573145), np.float64(0.25953848465342355), np.float64(0.21076136593758338), np.float64(0.20530969989014025), np.float64(0.15383733013823686)]
[np.float64(0.31031773872638835), np.float64(0.4673577227181982), np.float64(0.8614508956624429), np.float64(1.0256280903373935), np.float64(0.5512750207996723)]


In [ ]:
lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_undefined_classification_subgroup)

print(lower_bounds_saits_undefined_classification_subgroup)
print(upper_bounds_saits_undefined_classification_subgroup)

#### Mean values of lower bound and upper bound

In [80]:
mean_values_ci_saits_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_undefined_classification_subgroup, upper_bounds_saits_undefined_classification_subgroup)

print(mean_values_ci_saits_undefined_classification_subgroup)

[np.float64(0.23421292304635094), np.float64(0.37247299987905996), np.float64(0.5454084261161704), np.float64(0.637954897636116), np.float64(0.3888500314957763)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [81]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_low_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [82]:
means_bootstraps_saits_low_weight_classification_subgroup, standards_deviations_saits_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_low_weight_classification_subgroup)

print(means_bootstraps_saits_low_weight_classification_subgroup)
print(standards_deviations_saits_low_weight_classification_subgroup)


[np.float64(0.419192760030637), np.float64(1.2255543646398814), np.float64(0.14291983394555102), np.float64(0.0), np.float64(1.321715852773789)]
[np.float64(0.32079167367789113), np.float64(1.0840216547777217), np.float64(0.10788873457291617), np.float64(0.0), np.float64(1.1140293841094702)]


#### Calculating lower bound and upper bound

In [83]:
lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_low_weight_classification_subgroup, means_bootstraps_saits_low_weight_classification_subgroup, standards_deviations_saits_low_weight_classification_subgroup)

print(lower_bounds_saits_low_weight_classification_subgroup)
print(upper_bounds_saits_low_weight_classification_subgroup)

[np.float64(-0.2095473669183553), np.float64(-0.8990890371859648), np.float64(-0.06853820015496603), np.float64(0.0), np.float64(-0.8617416178001107)]
[np.float64(1.0479328869796292), np.float64(3.350197766465728), np.float64(0.3543778680460681), np.float64(0.0), np.float64(3.5051733233476883)]


In [84]:
lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_low_weight_classification_subgroup)

print(lower_bounds_saits_low_weight_classification_subgroup)
print(upper_bounds_saits_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.08279013359884828)]
[np.float64(0.6646819662222312), np.float64(2.8082556571929875), np.float64(0.22436394653942268), np.float64(0.0), np.float64(3.889406149694499)]


#### Mean values of lower bound and upper bound

In [85]:
mean_values_ci_saits_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_low_weight_classification_subgroup, upper_bounds_saits_low_weight_classification_subgroup)

print(mean_values_ci_saits_low_weight_classification_subgroup)

[np.float64(0.3323409831111156), np.float64(1.4041278285964938), np.float64(0.11218197326971134), np.float64(0.0), np.float64(1.9860981416466736)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [86]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_normal_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [87]:
means_bootstraps_saits_normal_weight_classification_subgroup, standards_deviations_saits_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_normal_weight_classification_subgroup)

print(means_bootstraps_saits_normal_weight_classification_subgroup)
print(standards_deviations_saits_normal_weight_classification_subgroup)


[np.float64(0.25615486179799773), np.float64(0.4345667231738347), np.float64(0.7179507259295334), np.float64(0.4547656885539283), np.float64(0.4912205847312096)]
[np.float64(0.06570161799535126), np.float64(0.11310437306015453), np.float64(0.3647735158883224), np.float64(0.17073431295097174), np.float64(0.16791734094726787)]


#### Calculating lower bound and upper bound

In [88]:
lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_normal_weight_classification_subgroup, means_bootstraps_saits_normal_weight_classification_subgroup, standards_deviations_saits_normal_weight_classification_subgroup)

print(lower_bounds_saits_normal_weight_classification_subgroup)
print(upper_bounds_saits_normal_weight_classification_subgroup)

[np.float64(0.12738205680110054), np.float64(0.2128862254819495), np.float64(0.0030077722743722113), np.float64(0.12013258424483314), np.float64(0.16210864409483167)]
[np.float64(0.38492766679489493), np.float64(0.65624722086572), np.float64(1.4328936795846945), np.float64(0.7893987928630235), np.float64(0.8203325253675875)]


In [89]:
lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_normal_weight_classification_subgroup)

print(lower_bounds_saits_normal_weight_classification_subgroup)
print(upper_bounds_saits_normal_weight_classification_subgroup)

[np.float64(0.14394780642304672), np.float64(0.24233614179620225), np.float64(0.0), np.float64(0.13240536833900718), np.float64(0.22209913023711622)]
[np.float64(0.40089251105478246), np.float64(0.6785728296896159), np.float64(1.157952386236277), np.float64(0.8058805108952452), np.float64(0.8638998760243497)]


#### Mean values of lower bound and upper bound

In [90]:
mean_values_ci_saits_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_normal_weight_classification_subgroup, upper_bounds_saits_normal_weight_classification_subgroup)

print(mean_values_ci_saits_normal_weight_classification_subgroup)

[np.float64(0.27242015873891456), np.float64(0.46045448574290904), np.float64(0.5789761931181385), np.float64(0.46914293961712616), np.float64(0.542999503130733)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [91]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_overweight_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [92]:
means_bootstraps_saits_overweight_classification_subgroup, standards_deviations_saits_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_overweight_classification_subgroup)

print(means_bootstraps_saits_overweight_classification_subgroup)
print(standards_deviations_saits_overweight_classification_subgroup)


[np.float64(0.4447811398265328), np.float64(0.2281732920938964), np.float64(0.9534987123891254), np.float64(0.8805436038205038), np.float64(0.2082095976659256)]
[np.float64(0.19837691404549285), np.float64(0.04669845695302999), np.float64(0.21848610585221714), np.float64(0.565915141445688), np.float64(0.06313334428224854)]


#### Calculating lower bound and upper bound

In [93]:
lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_overweight_classification_subgroup, means_bootstraps_saits_overweight_classification_subgroup, standards_deviations_saits_overweight_classification_subgroup)

print(lower_bounds_saits_overweight_classification_subgroup)
print(upper_bounds_saits_overweight_classification_subgroup)

[np.float64(0.05596953293316881), np.float64(0.13664599833236354), np.float64(0.525273813796374), np.float64(-0.2286296917189351), np.float64(0.0844705166491507)]
[np.float64(0.8335927467198967), np.float64(0.31970058585542926), np.float64(1.381723610981877), np.float64(1.9897168993599426), np.float64(0.3319486786827005)]


In [94]:
lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_overweight_classification_subgroup)

print(lower_bounds_saits_overweight_classification_subgroup)
print(upper_bounds_saits_overweight_classification_subgroup)

[np.float64(0.12219005287360903), np.float64(0.14140085926105736), np.float64(0.5268744376187863), np.float64(0.09391822253292265), np.float64(0.11839014971166002)]
[np.float64(0.8851430023244788), np.float64(0.32486996928020934), np.float64(1.3766193630858523), np.float64(2.216669352782489), np.float64(0.3545895231207918)]


#### Mean values of lower bound and upper bound

In [95]:
mean_values_ci_saits_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_overweight_classification_subgroup, upper_bounds_saits_overweight_classification_subgroup)

print(mean_values_ci_saits_overweight_classification_subgroup)

[np.float64(0.503666527599044), np.float64(0.23313541427063333), np.float64(0.9517469003523193), np.float64(1.1552937876577059), np.float64(0.2364898364162259)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [96]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs1_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [97]:
means_bootstraps_saits_obs1_classification_subgroup, standards_deviations_saits_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs1_classification_subgroup)

print(means_bootstraps_saits_obs1_classification_subgroup)
print(standards_deviations_saits_obs1_classification_subgroup)


[np.float64(0.3041519818686167), np.float64(0.2039811162570178), np.float64(0.6188635711438332), np.float64(0.4450996994069029), np.float64(0.3402994613750062)]
[np.float64(0.08640159168104183), np.float64(0.06247857792523965), np.float64(0.3125972227616065), np.float64(0.2112433437702371), np.float64(0.11332047402537393)]


#### Calculating lower bound and upper bound

In [98]:
lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs1_classification_subgroup, means_bootstraps_saits_obs1_classification_subgroup, standards_deviations_saits_obs1_classification_subgroup)

print(lower_bounds_saits_obs1_classification_subgroup)
print(upper_bounds_saits_obs1_classification_subgroup)

[np.float64(0.13480797396683916), np.float64(0.08152535371826884), np.float64(0.006184272863840046), np.float64(0.031070353643424586), np.float64(0.11819541357426663)]
[np.float64(0.4734959897703942), np.float64(0.32643687879576677), np.float64(1.2315428694238264), np.float64(0.8591290451703812), np.float64(0.5624035091757458)]


In [99]:
lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs1_classification_subgroup)

print(lower_bounds_saits_obs1_classification_subgroup)
print(upper_bounds_saits_obs1_classification_subgroup)

[np.float64(0.15345486451996385), np.float64(0.08901652278543665), np.float64(0.0), np.float64(0.0), np.float64(0.1582272321940467)]
[np.float64(0.48931622869693325), np.float64(0.33763213707655576), np.float64(1.2406693374067592), np.float64(0.6809092237281027), np.float64(0.5927917919845908)]


#### Mean values of lower bound and upper bound

In [100]:
mean_values_ci_saits_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs1_classification_subgroup, upper_bounds_saits_obs1_classification_subgroup)

print(mean_values_ci_saits_obs1_classification_subgroup)

[np.float64(0.32138554660844854), np.float64(0.2133243299309962), np.float64(0.6203346687033796), np.float64(0.34045461186405135), np.float64(0.3755095120893187)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [101]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs2_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [102]:
means_bootstraps_saits_obs2_classification_subgroup, standards_deviations_saits_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs2_classification_subgroup)

print(means_bootstraps_saits_obs2_classification_subgroup)
print(standards_deviations_saits_obs2_classification_subgroup)


[np.float64(1.3205242112100037), np.float64(0.16000733207266188), np.float64(0.0016255406763269627), np.float64(0.0), np.float64(0.20795988252884254)]
[np.float64(1.039591546742933), np.float64(0.04156486730591007), np.float64(0.0012448523688900867), np.float64(0.0), np.float64(0.1405700467028575)]


#### Calculating lower bound and upper bound

In [103]:
lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs2_classification_subgroup, means_bootstraps_saits_obs2_classification_subgroup, standards_deviations_saits_obs2_classification_subgroup)

print(lower_bounds_saits_obs2_classification_subgroup)
print(upper_bounds_saits_obs2_classification_subgroup)

[np.float64(-0.717037779038433), np.float64(0.07854168913089175), np.float64(-0.000814325132766977), np.float64(0.0), np.float64(-0.06755234631387155)]
[np.float64(3.3580862014584403), np.float64(0.241472975014432), np.float64(0.004065406485420902), np.float64(0.0), np.float64(0.48347211137155666)]


In [104]:
lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs2_classification_subgroup)

print(lower_bounds_saits_obs2_classification_subgroup)
print(upper_bounds_saits_obs2_classification_subgroup)

[np.float64(0.0), np.float64(0.08676975530664142), np.float64(0.0), np.float64(0.0), np.float64(0.05830122010408454)]
[np.float64(3.5585250031408417), np.float64(0.25086778695880807), np.float64(0.00257885882019196), np.float64(0.0), np.float64(0.5344201694987715)]


#### Mean values of lower bound and upper bound

In [105]:
mean_values_ci_saits_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs2_classification_subgroup, upper_bounds_saits_obs2_classification_subgroup)

print(mean_values_ci_saits_obs2_classification_subgroup)

[np.float64(1.7792625015704209), np.float64(0.16881877113272475), np.float64(0.00128942941009598), np.float64(0.0), np.float64(0.296360694801428)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [106]:
indices = [32, 0, 6, 31, 5]

results_bootstrap_saits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_saits_obs3_classification_subgroup.append(toolkits.bootstrap(testing_ae_saits_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [107]:
means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_saits_obs3_classification_subgroup)

print(means_bootstraps_saits_obs3_classification_subgroup)
print(standards_deviations_saits_obs3_classification_subgroup)


[np.float64(1.1812910869641466), np.float64(0.19666861434408842), np.float64(0.0), np.float64(0.24446362112376532), np.float64(0.28186422520699844)]
[np.float64(0.8265505067483679), np.float64(0.05429070224681887), np.float64(0.0), np.float64(0.18379005114393437), np.float64(0.14798304940421692)]


#### Calculating lower bound and upper bound

In [108]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_saits_obs3_classification_subgroup, means_bootstraps_saits_obs3_classification_subgroup, standards_deviations_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(-0.43871813766598544), np.float64(0.09026079324493565), np.float64(0.0), np.float64(-0.11575825983512059), np.float64(-0.008177221947678226)]
[np.float64(2.8013003115942787), np.float64(0.3030764354432412), np.float64(0.0), np.float64(0.6046855020826513), np.float64(0.5719056723616751)]


In [109]:
lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_saits_obs3_classification_subgroup)

print(lower_bounds_saits_obs3_classification_subgroup)
print(upper_bounds_saits_obs3_classification_subgroup)

[np.float64(0.1026794807332443), np.float64(0.10886157947913234), np.float64(0.0), np.float64(0.0), np.float64(0.04363921474152539)]
[np.float64(3.000223714404996), np.float64(0.3208344032217549), np.float64(0.0), np.float64(0.38263871132432054), np.float64(0.5965402930578486)]


#### Mean values of lower bound and upper bound

In [110]:
mean_values_ci_saits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_saits_obs3_classification_subgroup, upper_bounds_saits_obs3_classification_subgroup)

print(mean_values_ci_saits_obs3_classification_subgroup)

[np.float64(1.5514515975691203), np.float64(0.2148479913504436), np.float64(0.0), np.float64(0.19131935566216027), np.float64(0.320089753899687)]


## BRITS

### Calculate MAE and AE

In [111]:
testing_mae_brits_variables_standard, testing_ae_brits_variables_standard = toolkits.calculate_mae(brits_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for BRITS model

#### Difference between gender group

In [112]:
diff_brits_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 1, 2, variables)
diff_brits_gender_subgroup

[('Cholesterol', 0.33467024143836643),
 ('AST', 0.1788895577767497),
 ('TroponinI', 0.1752692538746189),
 ('Creatinine', 0.15555507415890046),
 ('Lactate', 0.09616951800510165)]

#### Difference between age group

In [113]:
diff_brits_age_subgroup = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 4, 5, variables)
diff_brits_age_subgroup

[('ALP', 0.44556706439740035),
 ('TroponinI', 0.38681949447134645),
 ('Bilirubin', 0.2381043500110501),
 ('Cholesterol', 0.22772041607621174),
 ('Albumin', 0.1383329640887001)]

#### Difference between ICUType group

In [114]:
diff_brits_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 7, variables)
diff_brits_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 8, variables)
diff_brits_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 6, 9, variables)
diff_brits_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 7, 8, variables)
diff_brits_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 7, 9, variables)
diff_brits_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 8, 9, variables)

In [115]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_brits_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_brits_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_brits_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_brits_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_brits_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_brits_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.964994104713767), ('TroponinI', 0.9581594648038441), ('TroponinT', 0.5486699277361194), ('Platelets', 0.25189168788950184), ('AST', 0.2412410795021756)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 1.0707510658734702), ('ALP', 0.9106143647131879), ('TroponinI', 0.7970524014883568), ('TroponinT', 0.4839761837292856), ('Cholesterol', 0.25934745989883623)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.634311116208685), ('TroponinT', 0.5409005934508104), ('Bilirubin', 0.2897210390508551), ('ALP', 0.26571690256150676), ('Cholesterol', 0.23623934982112949)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.030567257093563), ('ALP', 0.8822108131066266), ('Cholesterol', 0.7056466448149308), ('BUN', 0.33074672831006546), ('Platelets', 0.31153489724515043)]
----------------------------
Difere

In [116]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[0])
print(variables[6])
print(variables[31])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
ALP
Cholesterol
TroponinI
TroponinT


#### Difference between BMI group

In [117]:
diff_brits_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 11, variables)
diff_brits_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 12, variables)
diff_brits_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 13, variables)
diff_brits_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 14, variables)
diff_brits_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 15, variables)
diff_brits_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 10, 16, variables)

diff_brits_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 12, variables)
diff_brits_classification_low_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 13, variables)
diff_brits_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 14, variables)
diff_brits_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 15, variables)
diff_brits_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 11, 16, variables)

diff_brits_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 13, variables)
diff_brits_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 14, variables)
diff_brits_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 15, variables)
diff_brits_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 12, 16, variables)

diff_brits_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 14, variables)
diff_brits_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 15, variables)
diff_brits_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 13, 16, variables)


diff_brits_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 14, 15, variables)
diff_brits_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 14, 16, variables)

diff_brits_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_brits_variables_standard, 15, 16, variables)

In [118]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_brits_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_brits_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_brits_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_brits_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_brits_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_brits_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.9340446883544171), ('Bilirubin', 1.0634205222465751), ('TroponinI', 0.8461866751277154), ('Cholesterol', 0.4642718057586309), ('Lactate', 0.4329340274633583)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('SaO2', 0.39563652008480243), ('Cholesterol', 0.3133469070406917), ('TroponinI', 0.28119857470091913), ('Platelets', 0.21973579245745867), ('ALP', 0.14108823047399421)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.9122714195538194), ('SaO2', 0.22161477416310832), ('Bilirubin', 0.21564253468390476), ('TroponinI', 0.20102397158068586), ('ALT', 0.195959972616804)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8860429007278312), ('TroponinI', 0.2779627718329166), ('SaO2', 0.26307064437502575), ('Albumin', 0.2194660837

In [119]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_brits_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_brits_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_brits_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_brits_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_brits_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.792956457880423), ('Bilirubin', 0.951330519054483), ('Cholesterol', 0.7776187127993226), ('SaO2', 0.6685476737972635), ('TroponinI', 0.5649881004267963)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 2.0608654148254533), ('Cholesterol', 1.3765432253124503), ('Bilirubin', 1.27906305693048), ('TroponinI', 1.0472106467084012), ('Glucose', 0.40732195589248754)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 2.03388782015009), ('Bilirubin', 1.0689277687310215), ('Ph', 0.8810842117518044), ('TroponinI', 0.5682239032947988), ('Glucose', 0.561052165137238)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 2.0833439707474084), ('Bilirubin', 1.1899083597278504), ('TroponinT', 0.8194450706632078), ('AST', 0.44272371026975965), ('HCO3', 0.43447019436156575)]

In [120]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_brits_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_brits_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_brits_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_brits_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('SaO2', 0.6172512942479107), ('Cholesterol', 0.5989245125131277), ('TroponinI', 0.482222546281605), ('Bilirubin', 0.32773253787599693), ('ALT', 0.27363285234568174)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8903667411722032), ('SaO2', 0.6587071644598281), ('Cholesterol', 0.36517238501263594), ('Albumin', 0.29004740656919503), ('ALP', 0.24093136226966733)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.2204323157755508), ('Cholesterol', 0.6885114670116885), ('TroponinI', 0.5649881004267963), ('SaO2', 0.4915174493800929), ('ALT', 0.3479177650511717)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('TroponinT', 0.9141402591111647), ('Cholesterol', 0.8876134062748404), ('ALT', 0.8550561698787857), ('SaO2', 0.5055719791939934), ('ALP', 0.3959

In [121]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_brits_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_brits_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_brits_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 0.9640968975257637), ('Ph', 0.8905324608937575), ('TroponinI', 0.4789867434136025), ('ALT', 0.2822848044403897), ('Bilirubin', 0.21013528819945848)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('Cholesterol', 1.2874359795248163), ('TroponinI', 1.0472106467084012), ('TroponinT', 1.026222109628948), ('AST', 0.2181718782219345), ('BUN', 0.19186306569181605)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.4865379187879681), ('TroponinT', 0.719930052964562), ('TroponinI', 0.6550987903762581), ('ALT', 0.581423317533104), ('Glucose', 0.23574108148249084)]
----------------------------


In [122]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_brits_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_brits_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.1935023183719276), ('Ph', 0.8867898200352354), ('TroponinI', 0.5682239032947988), ('ALT', 0.3565697171458797), ('Cholesterol', 0.32333908199905265)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Ph', 0.8897412969673524), ('TroponinT', 0.8872102617075415), ('ALT', 0.8637081219734937), ('Cholesterol', 0.5224410212622045), ('Glucose', 0.3894712907272413)]
----------------------------


In [123]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_brits_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('ALT', 0.507138404827614), ('TroponinI', 0.3921118563321432), ('TroponinT', 0.3062920566643861), ('HCO3', 0.23358523804099007), ('Glucose', 0.20095365530478204)]
----------------------------


In [124]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[0])
print(variables[6])
print(variables[32])
print(variables[31])
print(variables[5])

Top 5 variáveis BMI
-----------------------
ALP
Cholesterol
TroponinT
TroponinI
Bilirubin


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [125]:
indices = [6, 2, 31, 7, 16]

results_bootstrap_brits_female = []

for i in indices:
       results_bootstrap_brits_female.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [126]:
means_bootstraps_brits_female,standards_deviations_brits_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_female)

print(means_bootstraps_brits_female)
print(standards_deviations_brits_female)


[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]
[np.float64(0.3381506628642691), np.float64(0.08322467917877542), np.float64(0.07193090333728737), np.float64(0.02436604729618584), np.float64(0.03851788865527756)]


#### Calculating lower bound and upper bound

In [127]:
lower_bounds_brits_female, upper_bounds_brits_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_female, means_bootstraps_brits_female,standards_deviations_brits_female)

print(lower_bounds_brits_female)
print(upper_bounds_brits_female)

[np.float64(0.5235200070760525), np.float64(0.1366787734720611), np.float64(0.3825933708508273), np.float64(0.20450627248860936), np.float64(0.3499327009241326)]
[np.float64(1.849046248200679), np.float64(0.46291352110266176), np.float64(0.6645573306838577), np.float64(0.30001942278085697), np.float64(0.5009200499738685)]


In [128]:
lower_bounds_brits_female, upper_bounds_brits_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_female)

print(lower_bounds_brits_female)
print(upper_bounds_brits_female)

[np.float64(0.4968807905077575), np.float64(0.15777963365890407), np.float64(0.38256497348656243), np.float64(0.20792187509631285), np.float64(0.3545808886663554)]
[np.float64(1.839347804995353), np.float64(0.482637756192842), np.float64(0.6661266330063176), np.float64(0.3031409155244055), np.float64(0.5060578657766025)]


#### Mean values of lower bound and upper bound


In [129]:
mean_values_ci_brits_female = toolkits.calc_mean_values_ci(lower_bounds_brits_female, upper_bounds_brits_female)

print(mean_values_ci_brits_female)

[np.float64(1.1681142977515553), np.float64(0.320208694925873), np.float64(0.52434580324644), np.float64(0.2555313953103592), np.float64(0.43031937722147895)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [130]:
indices = [6, 2, 31, 7, 16]

results_bootstrap_brits_male = []

for i in indices:
       results_bootstrap_brits_male.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [131]:
means_bootstraps_brits_male,standards_deviations_brits_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_male)

print(means_bootstraps_brits_male)
print(standards_deviations_brits_male)


[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]
[np.float64(0.17358454405745072), np.float64(0.025632322705534415), np.float64(0.2124414858781128), np.float64(0.03724148791663912), np.float64(0.04259881898681301)]


#### Calculating lower bound and upper bound

In [132]:
lower_bounds_brits_male, upper_bounds_brits_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_male, means_bootstraps_brits_male,standards_deviations_brits_male)

print(lower_bounds_brits_male)
print(upper_bounds_brits_male)

[np.float64(0.5129891453019042), np.float64(0.07149955530644256), np.float64(0.2837910673267833), np.float64(0.33405772637289466), np.float64(0.4386664730964766)]
[np.float64(1.1934280545527236), np.float64(0.17197641399235403), np.float64(1.1165463896133345), np.float64(0.4800416764674872), np.float64(0.6056507750926656)]


In [134]:
lower_bounds_brits_male, upper_bounds_brits_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_male)

print(lower_bounds_brits_male)
print(upper_bounds_brits_male)

[np.float64(0.5279936705737495), np.float64(0.07721704416107987), np.float64(0.3292561099811831), np.float64(0.33737493991402795), np.float64(0.44382017231433574)]
[np.float64(1.2043504550964836), np.float64(0.17693865508789547), np.float64(1.1598741699157562), np.float64(0.4832001133644307), np.float64(0.6097816615527988)]


#### Mean values of lower bound and upper bound


In [135]:
mean_values_ci_brits_male = toolkits.calc_mean_values_ci(lower_bounds_brits_male, upper_bounds_brits_male)

print(mean_values_ci_brits_male)

[np.float64(0.8661720628351166), np.float64(0.12707784962448768), np.float64(0.7445651399484696), np.float64(0.4102875266392293), np.float64(0.5268009169335672)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [136]:
indices = [0, 31, 5, 6, 3]

results_bootstrap_brits_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_brits_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [137]:
means_bootstraps_brits_more_than_or_equal_to_65_subgroup,standards_deviations_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_brits_more_than_or_equal_to_65_subgroup)
print(standards_deviations_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.9260057824625499), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]
[np.float64(0.14022222850550667), np.float64(0.2902139592129938), np.float64(0.025161049870449845), np.float64(0.08795901770065773), np.float64(0.06166376517595487)]


#### Calculating lower bound and upper bound


In [138]:
lower_bounds_brits_more_than_or_equal_to_65_subgroup, upper_bounds_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_more_than_or_equal_to_65_subgroup, means_bootstraps_brits_more_than_or_equal_to_65_subgroup,standards_deviations_brits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_brits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_brits_more_than_or_equal_to_65_subgroup)


[np.float64(0.3602797122452329), np.float64(0.3571968745943058), np.float64(0.18962243507704596), np.float64(0.491014463045211), np.float64(0.45334176332908904)]
[np.float64(0.9099407476507105), np.float64(1.4948146903307942), np.float64(0.2882519381956417), np.float64(0.8358074766628315), np.float64(0.6950592811211025)]


In [139]:
lower_bounds_brits_more_than_or_equal_to_65_subgroup, upper_bounds_brits_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_more_than_or_equal_to_65_subgroup)

print(lower_bounds_brits_more_than_or_equal_to_65_subgroup)
print(upper_bounds_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.3870394478539412), np.float64(0.46568194507300736), np.float64(0.19292104005655877), np.float64(0.48875435337841105), np.float64(0.45853042514994774)]
[np.float64(0.9343760347819378), np.float64(1.5660366563288135), np.float64(0.2921663838336204), np.float64(0.8327365001093664), np.float64(0.7004820435889465)]


#### Mean values of lower bound and upper bound

In [140]:
mean_values_ci_brits_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_more_than_or_equal_to_65_subgroup , upper_bounds_brits_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_brits_more_than_or_equal_to_65_subgroup)

[np.float64(0.6607077413179394), np.float64(1.0158593007009105), np.float64(0.2425437119450896), np.float64(0.6607454267438887), np.float64(0.5795062343694471)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [141]:
indices = [0, 31, 5, 6, 3]

results_bootstrap_brits_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_brits_less_than_65_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [142]:
means_bootstraps_brits_less_than_65_subgroup,standards_deviations_brits_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_less_than_65_subgroup)

print(means_bootstraps_brits_less_than_65_subgroup)
print(standards_deviations_brits_less_than_65_subgroup)


[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]
[np.float64(0.4337083267621267), np.float64(0.06965814575847884), np.float64(0.08711583404267569), np.float64(0.6101374424469065), np.float64(0.06334923482333744)]


#### Calculating lower bound and upper bound

In [143]:
lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_less_than_65_subgroup, means_bootstraps_brits_less_than_65_subgroup, standards_deviations_brits_less_than_65_subgroup)

print(lower_bounds_brits_less_than_65_subgroup)
print(upper_bounds_brits_less_than_65_subgroup)

[np.float64(0.23705163755647063), np.float64(0.40032284839995347), np.float64(0.30726428110681003), np.float64(-0.41507045428487355), np.float64(0.5887783496794352)]
[np.float64(1.9371570380542658), np.float64(0.6733777622328736), np.float64(0.6487520755204355), np.float64(1.9766243713457599), np.float64(0.8371027870832592)]


In [144]:
lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_less_than_65_subgroup)

print(lower_bounds_brits_less_than_65_subgroup)
print(upper_bounds_brits_less_than_65_subgroup)

[np.float64(0.40960047420909507), np.float64(0.37965413188890884), np.float64(0.3230237721847463), np.float64(0.0), np.float64(0.592142896632086)]
[np.float64(2.0765479628767127), np.float64(0.6430158128184212), np.float64(0.6636826654971765), np.float64(1.640032252107956), np.float64(0.8413776344626777)]


#### Mean values of lower bound and upper bound

In [145]:
mean_values_ci_brits_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_less_than_65_subgroup, upper_bounds_brits_less_than_65_subgroup)

print(mean_values_ci_brits_less_than_65_subgroup)

[np.float64(1.2430742185429038), np.float64(0.5113349723536651), np.float64(0.4933532188409614), np.float64(0.820016126053978), np.float64(0.7167602655473818)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [146]:
indices = [28, 0, 6, 31, 32]

results_bootstrap_brits_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_1_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [147]:
means_bootstraps_brits_ICUType_1_subgroup, standards_deviations_brits_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_1_subgroup)

print(means_bootstraps_brits_ICUType_1_subgroup)
print(standards_deviations_brits_ICUType_1_subgroup)


[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547066), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]
[np.float64(0.17812570383563214), np.float64(0.03508402734675316), np.float64(0.2710993234914266), np.float64(0.37708432885457466), np.float64(0.15436752793597142)]


#### Calculating lower bound and upper bound

In [148]:
lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_1_subgroup, means_bootstraps_brits_ICUType_1_subgroup, standards_deviations_brits_ICUType_1_subgroup)

print(lower_bounds_brits_ICUType_1_subgroup)
print(upper_bounds_brits_ICUType_1_subgroup)

[np.float64(0.1735625173126043), np.float64(0.1565126112697423), np.float64(0.437748337678337), np.float64(0.6022929596856587), np.float64(0.41865583030294223)]
[np.float64(0.8718024457899787), np.float64(0.29403947133425135), np.float64(1.5004381582310762), np.float64(2.080436367064507), np.float64(1.0237654205769116)]


In [149]:
lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_1_subgroup)

print(lower_bounds_brits_ICUType_1_subgroup)
print(upper_bounds_brits_ICUType_1_subgroup)

[np.float64(0.28256831897763984), np.float64(0.15992531714336428), np.float64(0.4939563545686042), np.float64(0.6875523418113644), np.float64(0.4421847571220248)]
[np.float64(0.9443799897801524), np.float64(0.29705740948586223), np.float64(1.5643299653136569), np.float64(2.134296145454983), np.float64(1.0443018856551611)]


#### Mean values of lower bound and upper bound

In [150]:
mean_values_ci_brits_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_1_subgroup, upper_bounds_brits_ICUType_1_subgroup)

print(mean_values_ci_brits_ICUType_1_subgroup)

[np.float64(0.6134741543788961), np.float64(0.22849136331461325), np.float64(1.0291431599411305), np.float64(1.4109242436331737), np.float64(0.743243321388593)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [151]:
indices = [28, 0, 6, 31, 32]

results_bootstrap_brits_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_2_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [152]:
means_bootstraps_brits_ICUType_2_subgroup, standards_deviations_brits_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_2_subgroup)

print(means_bootstraps_brits_ICUType_2_subgroup)
print(standards_deviations_brits_ICUType_2_subgroup)


[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.3769132654783525), np.float64(0.17130476566918515)]
[np.float64(0.24746020204511532), np.float64(0.02844864967364565), np.float64(0.0), np.float64(0.12241202033633213), np.float64(0.04748266503243493)]


#### Calculating lower bound and upper bound

In [153]:
lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_2_subgroup, means_bootstraps_brits_ICUType_2_subgroup, standards_deviations_brits_ICUType_2_subgroup)

print(lower_bounds_brits_ICUType_2_subgroup)
print(upper_bounds_brits_ICUType_2_subgroup)

[np.float64(0.07787625683221377), np.float64(0.19778635374686335), np.float64(0.0), np.float64(0.13699011434435687), np.float64(0.07824045231563329)]
[np.float64(1.0479024240630759), np.float64(0.3093030112851486), np.float64(0.0), np.float64(0.6168364166123481), np.float64(0.264369079022737)]


In [154]:
lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_2_subgroup)

#### Mean values of lower bound and upper bound

In [155]:
mean_values_ci_brits_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_2_subgroup, upper_bounds_brits_ICUType_2_subgroup)

print(mean_values_ci_brits_ICUType_2_subgroup)

[np.float64(0.6996836620650728), np.float64(0.25573667275300277), np.float64(0.0), np.float64(0.378342729883637), np.float64(0.18299999636214556)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [156]:
indices = [28, 0, 6, 31, 32]

results_bootstrap_brits_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_3_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [157]:
means_bootstraps_brits_ICUType_3_subgroup, standards_deviations_brits_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_3_subgroup)

print(means_bootstraps_brits_ICUType_3_subgroup)
print(standards_deviations_brits_ICUType_3_subgroup)


[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]
[np.float64(0.4559261164801034), np.float64(0.45097433675337983), np.float64(0.1878586172758027), np.float64(0.06133680568713781), np.float64(0.03548543506753726)]


#### Calculating lower bound and upper bound

In [158]:
lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_3_subgroup, means_bootstraps_brits_ICUType_3_subgroup, standards_deviations_brits_ICUType_3_subgroup)

print(lower_bounds_brits_ICUType_3_subgroup)
print(upper_bounds_brits_ICUType_3_subgroup)

[np.float64(0.6996377293253487), np.float64(0.25263761572853904), np.float64(0.3276231392998342), np.float64(0.4209399971546227), np.float64(0.16790862349933133)]
[np.float64(2.4868352651497814), np.float64(2.020424531705464), np.float64(1.0640153873919687), np.float64(0.661375857301666), np.float64(0.30700897291554674)]


In [159]:
lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_3_subgroup)

print(lower_bounds_brits_ICUType_3_subgroup)
print(upper_bounds_brits_ICUType_3_subgroup)

[np.float64(0.7811074732923107), np.float64(0.42602553654535447), np.float64(0.3370785960436855), np.float64(0.4225742806604089), np.float64(0.17750753035417519)]
[np.float64(2.5559583373020067), np.float64(2.1449893682804095), np.float64(1.1317699079657495), np.float64(0.664792763097355), np.float64(0.31561312193846874)]


#### Mean values of lower bound and upper bound

In [160]:
mean_values_ci_brits_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_3_subgroup, upper_bounds_brits_ICUType_3_subgroup)

print(mean_values_ci_brits_ICUType_3_subgroup)

[np.float64(1.6685329052971587), np.float64(1.285507452412882), np.float64(0.7344242520047175), np.float64(0.543683521878882), np.float64(0.24656032614632195)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [161]:
indices = [28, 0, 6, 31, 32]

results_bootstrap_brits_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_brits_ICUType_4_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [162]:
means_bootstraps_brits_ICUType_4_subgroup, standards_deviations_brits_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_ICUType_4_subgroup)

print(means_bootstraps_brits_ICUType_4_subgroup)
print(standards_deviations_brits_ICUType_4_subgroup)


[np.float64(0.44306473111336137), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]
[np.float64(0.0935566892280583), np.float64(0.08074364451634707), np.float64(0.2681495472441715), np.float64(0.2724504216824265), np.float64(0.022189091660584512)]


#### Calculating lower bound and upper bound

In [163]:
lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_ICUType_4_subgroup, means_bootstraps_brits_ICUType_4_subgroup, standards_deviations_brits_ICUType_4_subgroup)

print(lower_bounds_brits_ICUType_4_subgroup)
print(upper_bounds_brits_ICUType_4_subgroup)

[np.float64(0.25969698971356064), np.float64(0.33294655262192163), np.float64(0.2031662976673827), np.float64(0.1584331675516304), np.float64(0.13677304273715427)]
[np.float64(0.626432472513162), np.float64(0.6494558230870122), np.float64(1.2542932078059783), np.float64(1.2264191956922437), np.float64(0.22375268374596166)]


In [164]:
lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_ICUType_4_subgroup)

print(lower_bounds_brits_ICUType_4_subgroup)
print(upper_bounds_brits_ICUType_4_subgroup)

[np.float64(0.2865496885836767), np.float64(0.34462669540294605), np.float64(0.22052486820293912), np.float64(0.031016071654553733), np.float64(0.13675082536379823)]
[np.float64(0.6531075788873016), np.float64(0.6642258024673423), np.float64(1.2816077240763408), np.float64(1.1697815665096722), np.float64(0.22392540460300034)]


#### Mean values of lower bound and upper bound

In [165]:
mean_values_ci_brits_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_ICUType_4_subgroup, upper_bounds_brits_ICUType_4_subgroup)

print(mean_values_ci_brits_ICUType_4_subgroup)

[np.float64(0.46982863373548917), np.float64(0.5044262489351442), np.float64(0.7510662961396399), np.float64(0.600398819082113), np.float64(0.1803381149833993)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [166]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_undefined_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [167]:
means_bootstraps_brits_undefined_classification_subgroup, standards_deviations_brits_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_undefined_classification_subgroup)

print(means_bootstraps_brits_undefined_classification_subgroup)
print(standards_deviations_brits_undefined_classification_subgroup)


[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]
[np.float64(0.05986934359644469), np.float64(0.17840974136243826), np.float64(0.044640773548291844), np.float64(0.2853216956527157), np.float64(0.12549413739177268)]


#### Calculating lower bound and upper bound

In [168]:
lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_undefined_classification_subgroup, means_bootstraps_brits_undefined_classification_subgroup, standards_deviations_brits_undefined_classification_subgroup)

print(lower_bounds_brits_undefined_classification_subgroup)
print(upper_bounds_brits_undefined_classification_subgroup)

[np.float64(0.28951948015518403), np.float64(0.22394384394790923), np.float64(0.18842469528371564), np.float64(0.28379016616066055), np.float64(0.16723269590166293)]
[np.float64(0.5242029946093546), np.float64(0.9232971790708792), np.float64(0.3634133120770363), np.float64(1.4022306611351032), np.float64(0.6591606750192545)]


In [169]:
lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_undefined_classification_subgroup)

print(lower_bounds_brits_undefined_classification_subgroup)
print(upper_bounds_brits_undefined_classification_subgroup)

[np.float64(0.2979609801580593), np.float64(0.24726862471409536), np.float64(0.2040999395523388), np.float64(0.40619839098934485), np.float64(0.2171298071300369)]
[np.float64(0.5319916259081999), np.float64(0.9465467461362519), np.float64(0.3763968339372512), np.float64(1.4766778181071094), np.float64(0.6956717509434502)]


#### Mean values of lower bound and upper bound

In [170]:
mean_values_ci_brits_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_undefined_classification_subgroup, upper_bounds_brits_undefined_classification_subgroup)

print(mean_values_ci_brits_undefined_classification_subgroup)

[np.float64(0.4149763030331296), np.float64(0.5969076854251736), np.float64(0.29024838674479503), np.float64(0.9414381045482272), np.float64(0.4564007790367436)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [171]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_low_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [172]:
means_bootstraps_brits_low_weight_classification_subgroup, standards_deviations_brits_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_low_weight_classification_subgroup)

print(means_bootstraps_brits_low_weight_classification_subgroup)
print(standards_deviations_brits_low_weight_classification_subgroup)


[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032636), np.float64(0.0), np.float64(1.4606367473586466)]
[np.float64(1.666676645356208), np.float64(0.05309602591127397), np.float64(0.3139178166078663), np.float64(0.0), np.float64(1.2916342659860158)]


#### Calculating lower bound and upper bound

In [173]:
lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_low_weight_classification_subgroup, means_bootstraps_brits_low_weight_classification_subgroup, standards_deviations_brits_low_weight_classification_subgroup)

print(lower_bounds_brits_low_weight_classification_subgroup)
print(upper_bounds_brits_low_weight_classification_subgroup)

[np.float64(-1.2543455052071963), np.float64(-0.03473297672088424), np.float64(-0.2048614788536039), np.float64(0.0), np.float64(-1.070919895171773)]
[np.float64(5.278906892337211), np.float64(0.17339962029572073), np.float64(1.0256737504601312), np.float64(0.0), np.float64(3.9921933898890662)]


In [174]:
lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_low_weight_classification_subgroup)

print(lower_bounds_brits_low_weight_classification_subgroup)
print(upper_bounds_brits_low_weight_classification_subgroup)

[np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.020133205116918334)]
[np.float64(4.413896351691468), np.float64(0.10999469347559115), np.float64(0.6505204688677284), np.float64(0.0), np.float64(4.452386787366754)]


#### Mean values of lower bound and upper bound

In [175]:
mean_values_ci_brits_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_low_weight_classification_subgroup, upper_bounds_brits_low_weight_classification_subgroup)

print(mean_values_ci_brits_low_weight_classification_subgroup)

[np.float64(2.206948175845734), np.float64(0.05499734673779558), np.float64(0.3252602344338642), np.float64(0.0), np.float64(2.236259996241836)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [176]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_normal_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [177]:
means_bootstraps_brits_normal_weight_classification_subgroup, standards_deviations_brits_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_normal_weight_classification_subgroup)

print(means_bootstraps_brits_normal_weight_classification_subgroup)
print(standards_deviations_brits_normal_weight_classification_subgroup)


[np.float64(0.548686639105505), np.float64(0.7625897872503594), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]
[np.float64(0.12623025707018218), np.float64(0.3731308327837512), np.float64(0.07341434562099826), np.float64(0.089947135630567), np.float64(0.2013724566774524)]


#### Calculating lower bound and upper bound

In [178]:
lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_normal_weight_classification_subgroup, means_bootstraps_brits_normal_weight_classification_subgroup, standards_deviations_brits_normal_weight_classification_subgroup)

print(lower_bounds_brits_normal_weight_classification_subgroup)
print(upper_bounds_brits_normal_weight_classification_subgroup)

[np.float64(0.3012798814887154), np.float64(0.03126679347276984), np.float64(0.10543028800815332), np.float64(0.38859566088565467), np.float64(0.12860300267586205)]
[np.float64(0.7960933967222945), np.float64(1.4939127810279489), np.float64(0.3932092347396181), np.float64(0.7411819535825561), np.float64(0.9179685278081802)]


In [179]:
lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_normal_weight_classification_subgroup)

print(lower_bounds_brits_normal_weight_classification_subgroup)
print(upper_bounds_brits_normal_weight_classification_subgroup)

[np.float64(0.34098181461031984), np.float64(0.0), np.float64(0.13019003204955729), np.float64(0.3943044247772959), np.float64(0.20193025625589317)]
[np.float64(0.822822162147693), np.float64(1.1889972866535945), np.float64(0.41316823841970496), np.float64(0.7470447554607124), np.float64(0.9848085395794249)]


#### Mean values of lower bound and upper bound

In [180]:
mean_values_ci_brits_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_normal_weight_classification_subgroup, upper_bounds_brits_normal_weight_classification_subgroup)

print(mean_values_ci_brits_normal_weight_classification_subgroup)

[np.float64(0.5819019883790064), np.float64(0.5944986433267972), np.float64(0.2716791352346311), np.float64(0.5706745901190041), np.float64(0.5933693979176591)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [181]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_overweight_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [182]:
means_bootstraps_brits_overweight_classification_subgroup, standards_deviations_brits_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_overweight_classification_subgroup)

print(means_bootstraps_brits_overweight_classification_subgroup)
print(standards_deviations_brits_overweight_classification_subgroup)


[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082165), np.float64(0.1969866594273391)]
[np.float64(0.05690905629749502), np.float64(0.2748051570149171), np.float64(0.190452089816047), np.float64(0.5659200713279424), np.float64(0.06465085722820384)]


#### Calculating lower bound and upper bound

In [183]:
lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_overweight_classification_subgroup, means_bootstraps_brits_overweight_classification_subgroup, standards_deviations_brits_overweight_classification_subgroup)

print(lower_bounds_brits_overweight_classification_subgroup)
print(upper_bounds_brits_overweight_classification_subgroup)

[np.float64(0.16821060322753356), np.float64(0.9483102156492423), np.float64(0.07116983979697916), np.float64(-0.06774702542288913), np.float64(0.07027330769041853)]
[np.float64(0.3912900047020388), np.float64(2.0255266366794666), np.float64(0.8177283134366586), np.float64(2.150618890439322), np.float64(0.3237000111642596)]


In [184]:
lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_overweight_classification_subgroup)

print(lower_bounds_brits_overweight_classification_subgroup)
print(upper_bounds_brits_overweight_classification_subgroup)

[np.float64(0.18291051175428089), np.float64(0.9097202836560881), np.float64(0.14614286145696292), np.float64(0.3791864463768746), np.float64(0.10540658498615474)]
[np.float64(0.40407734173253634), np.float64(1.9873488893468048), np.float64(0.8647336127806425), np.float64(2.277614098374932), np.float64(0.3472386410647299)]


#### Mean values of lower bound and upper bound

In [185]:
mean_values_ci_brits_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_overweight_classification_subgroup, upper_bounds_brits_overweight_classification_subgroup)

print(mean_values_ci_brits_overweight_classification_subgroup)

[np.float64(0.29349392674340863), np.float64(1.4485345865014465), np.float64(0.5054382371188028), np.float64(1.3284002723759034), np.float64(0.22632261302544232)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [186]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs1_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [187]:
means_bootstraps_brits_obs1_classification_subgroup, standards_deviations_brits_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs1_classification_subgroup)

print(means_bootstraps_brits_obs1_classification_subgroup)
print(standards_deviations_brits_obs1_classification_subgroup)


[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]
[np.float64(0.05798396274724235), np.float64(0.25089896187847854), np.float64(0.08655489332963262), np.float64(0.19483046174517651), np.float64(0.12094499608671105)]


#### Calculating lower bound and upper bound

In [188]:
lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs1_classification_subgroup, means_bootstraps_brits_obs1_classification_subgroup, standards_deviations_brits_obs1_classification_subgroup)

print(lower_bounds_brits_obs1_classification_subgroup)
print(upper_bounds_brits_obs1_classification_subgroup)

[np.float64(0.19364783590608406), np.float64(0.0070376496942498035), np.float64(0.10522008285767387), np.float64(0.11111241585757914), np.float64(0.1681406336979566)]
[np.float64(0.42094079323709843), np.float64(0.9905435077748617), np.float64(0.44450903008124604), np.float64(0.8748337920812886), np.float64(0.6422363065785395)]


In [189]:
lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs1_classification_subgroup)

print(lower_bounds_brits_obs1_classification_subgroup)
print(upper_bounds_brits_obs1_classification_subgroup)

[np.float64(0.2017864229804127), np.float64(0.0), np.float64(0.12255724300388852), np.float64(0.0), np.float64(0.2152851960298434)]
[np.float64(0.4295377781988619), np.float64(1.0159483303577879), np.float64(0.4588269895582404), np.float64(0.621003118217197), np.float64(0.6770095865971394)]


#### Mean values of lower bound and upper bound

In [190]:
mean_values_ci_brits_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs1_classification_subgroup, upper_bounds_brits_obs1_classification_subgroup)

print(mean_values_ci_brits_obs1_classification_subgroup)

[np.float64(0.3156621005896373), np.float64(0.5079741651788939), np.float64(0.29069211628106445), np.float64(0.3105015591085985), np.float64(0.4461473913134914)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [191]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs2_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [192]:
means_bootstraps_brits_obs2_classification_subgroup, standards_deviations_brits_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs2_classification_subgroup)

print(means_bootstraps_brits_obs2_classification_subgroup)
print(standards_deviations_brits_obs2_classification_subgroup)


[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.3961240813101228), np.float64(0.0), np.float64(0.2858736875756382)]
[np.float64(0.049002577704985846), np.float64(0.09598273849535295), np.float64(1.0268585416236724), np.float64(0.0), np.float64(0.13790114187609237)]


#### Calculating lower bound and upper bound

In [193]:
lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs2_classification_subgroup, means_bootstraps_brits_obs2_classification_subgroup, standards_deviations_brits_obs2_classification_subgroup)

print(lower_bounds_brits_obs2_classification_subgroup)
print(upper_bounds_brits_obs2_classification_subgroup)

[np.float64(0.1605865061227179), np.float64(-0.062157550347901436), np.float64(-0.6164816774895991), np.float64(0.0), np.float64(0.015592416071548876)]
[np.float64(0.35267308102551326), np.float64(0.3140878708289346), np.float64(3.408729840109845), np.float64(0.0), np.float64(0.5561549590797275)]


In [194]:
lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs2_classification_subgroup)

print(lower_bounds_brits_obs2_classification_subgroup)
print(upper_bounds_brits_obs2_classification_subgroup)

[np.float64(0.15852749748120876), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.10914732115072498)]
[np.float64(0.35302742539780924), np.float64(0.1991019392632846), np.float64(3.5812912356313698), np.float64(0.0), np.float64(0.602445513613066)]


#### Mean values of lower bound and upper bound

In [195]:
mean_values_ci_brits_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs2_classification_subgroup, upper_bounds_brits_obs2_classification_subgroup)

print(mean_values_ci_brits_obs2_classification_subgroup)

[np.float64(0.25577746143950897), np.float64(0.0995509696316423), np.float64(1.7906456178156849), np.float64(0.0), np.float64(0.3557964173818955)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [196]:
indices = [0, 6, 32, 31, 5]

results_bootstrap_brits_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_brits_obs3_classification_subgroup.append(toolkits.bootstrap(testing_ae_brits_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [197]:
means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_brits_obs3_classification_subgroup)

print(means_bootstraps_brits_obs3_classification_subgroup)
print(standards_deviations_brits_obs3_classification_subgroup)


[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.20886002502599546)]
[np.float64(0.050281311239894225), np.float64(0.0), np.float64(0.8237478596037178), np.float64(0.19004214803335054), np.float64(0.12978497928121874)]


#### Calculating lower bound and upper bound

In [198]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_brits_obs3_classification_subgroup, means_bootstraps_brits_obs3_classification_subgroup, standards_deviations_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(0.05300461518183637), np.float64(0.0), np.float64(-0.4551198683898825), np.float64(-0.12823364717905072), np.float64(-0.045513860099470366)]
[np.float64(0.25010373343311976), np.float64(0.0), np.float64(2.773912405940605), np.float64(0.6167178842009423), np.float64(0.4632339101514613)]


In [199]:
lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_brits_obs3_classification_subgroup)

print(lower_bounds_brits_obs3_classification_subgroup)
print(upper_bounds_brits_obs3_classification_subgroup)

[np.float64(0.06726677849320917), np.float64(0.0), np.float64(0.15495381460601493), np.float64(0.0), np.float64(0.03269967207779254)]
[np.float64(0.2607674007481455), np.float64(0.0), np.float64(2.983221065811697), np.float64(0.3921118563324046), np.float64(0.487857401911803)]


#### Mean values of lower bound and upper bound

In [200]:
mean_values_ci_brits_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_brits_obs3_classification_subgroup, upper_bounds_brits_obs3_classification_subgroup)

print(mean_values_ci_brits_obs3_classification_subgroup)

[np.float64(0.16401708962067732), np.float64(0.0), np.float64(1.569087440208856), np.float64(0.1960559281662023), np.float64(0.26027853699479775)]


## USGAN

### Calculate MAE and AE

In [ ]:
testing_mae_usgan_variables_standard, testing_ae_usgan_variables_standard = toolkits.calculate_mae(usgan_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for USGAN model

#### Difference between gender group

In [ ]:
diff_usgan_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 1, 2, variables)
diff_usgan_gender_subgroup

[('Cholesterol', 0.11396848832179907),
 ('K', 0.07962183727222638),
 ('Bilirubin', 0.07273530011716128),
 ('SaO2', 0.06724924437847979),
 ('AST', 0.06357150812579351)]

#### Difference between age group

In [ ]:
diff_usgan_age_subgroup = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 4, 5, variables)
diff_usgan_age_subgroup

[('Cholesterol', 1.4080332966852902),
 ('TroponinI', 1.2745449415570849),
 ('ALP', 0.46489726331108006),
 ('SaO2', 0.14584230629740058),
 ('Bilirubin', 0.14438651035533773)]

#### Difference between ICUType group

In [ ]:
diff_usgan_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 7, variables)
diff_usgan_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 8, variables)
diff_usgan_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 6, 9, variables)
diff_usgan_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 7, 8, variables)
diff_usgan_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 7, 9, variables)
diff_usgan_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 8, 9, variables)

In [ ]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_usgan_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_usgan_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_usgan_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_usgan_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_usgan_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_usgan_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 1.968532838973082), ('TroponinT', 0.3139744935635816), ('TroponinI', 0.2476191651050459), ('Glucose', 0.17361770275953042), ('Temp', 0.15812872602883854)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('TroponinI', 1.054054323556795), ('ALP', 0.9588980980944308), ('SaO2', 0.9557847823462609), ('Cholesterol', 0.720808558519628), ('TroponinT', 0.3983027553119586)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.8469546316747383), ('Cholesterol', 0.4515708800192406), ('TroponinT', 0.43928515289325093), ('ALP', 0.2595151866425361), ('Glucose', 0.1775868477420237)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('Cholesterol', 2.68934139749271), ('TroponinI', 1.3016734886618408), ('ALP', 0.9781535556167317), ('SaO2', 0.8722191782384636), ('Temp', 0.2501181579286637)]
----------------------------
Diferença(MAE

In [ ]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[28])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
Cholesterol
TroponinI
ALP
SaO2
TroponinT


#### Difference between BMI group

In [ ]:
diff_usgan_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 11, variables)
diff_usgan_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 12, variables)
diff_usgan_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 13, variables)
diff_usgan_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 14, variables)
diff_usgan_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 15, variables)
diff_usgan_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 10, 16, variables)

diff_usgan_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 12, variables)
diff_usgan_classification_low_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 13, variables)
diff_usgan_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 14, variables)
diff_usgan_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 15, variables)
diff_usgan_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 11, 16, variables)

diff_usgan_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 13, variables)
diff_usgan_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 14, variables)
diff_usgan_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 15, variables)
diff_usgan_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 12, 16, variables)

diff_usgan_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 14, variables)
diff_usgan_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 15, variables)
diff_usgan_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 13, 16, variables)


diff_usgan_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 14, 15, variables)
diff_usgan_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 14, 16, variables)

diff_usgan_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_usgan_variables_standard, 15, 16, variables)

In [ ]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_usgan_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_usgan_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_usgan_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_usgan_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_usgan_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_usgan_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.9859620009058019), ('TroponinI', 1.3663260793164698), ('Bilirubin', 1.0400217434134755), ('Cholesterol', 0.9605290481335007), ('SaO2', 0.3826989561208938)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 1.5579458557188144), ('TroponinI', 0.8172833755393478), ('SaO2', 0.5748494084312364), ('Platelets', 0.11877231573919644), ('ALP', 0.10391918734958416)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.7813610757706175), ('TroponinI', 0.528914503705096), ('SaO2', 0.26335364804066497), ('TroponinT', 0.15046354582798027), ('Bilirubin', 0.13730852400659188)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Cholesterol', 1.113359877891227), ('Ph', 0.8795575969438973), ('TroponinI', 0.5672432528016587), ('Albumin', 0.2786

In [ ]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_usgan_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_usgan_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_usgan_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_usgan_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_usgan_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('TroponinI', 2.1836094548558176), ('ALP', 1.8820428135562177), ('Bilirubin', 1.0046745883347163), ('SaO2', 0.9575483645521302), ('Cholesterol', 0.5974168075853137)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 2.057108090815585), ('TroponinI', 1.8952405830215657), ('Cholesterol', 1.7418901239041182), ('Bilirubin', 1.1773302674200674), ('Glucose', 0.41411061578867114)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.9622075950694193), ('TroponinI', 1.9335693321181284), ('Bilirubin', 1.2290397495761565), ('Ph', 0.8756933509066762), ('Glucose', 0.4445408934992038)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 2.2096916889234652), ('Bilirubin', 1.1530946428930196), ('TroponinT', 0.5311563166725247), ('HCO3', 0.38165713898274684), ('BUN', 0.3429542758853

In [ ]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_usgan_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_usgan_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_usgan_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_usgan_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('Cholesterol', 2.339306931489432), ('SaO2', 0.8382030564719014), ('TroponinI', 0.28836887183425186), ('Lactate', 0.17658435332815797), ('ALP', 0.17506527725936732)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8895493110798054), ('SaO2', 0.8483770038896792), ('Cholesterol', 0.44458597782758735), ('Albumin', 0.3456592406793608), ('TroponinI', 0.25004012273768916)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinI', 2.1836094548558176), ('Cholesterol', 0.9187488642412167), ('SaO2', 0.7611965781906), ('TroponinT', 0.6853385828101566), ('ALP', 0.32764887536724757)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('Cholesterol', 3.416495052966329), ('TroponinT', 0.7593013271876037), ('SaO2', 0.6919062266464278), ('ALT', 0.6246717920178768), ('Albumin', 0

In [ ]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_usgan_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_usgan_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_usgan_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 1.8947209536618446), ('Ph', 0.8907999481313389), ('TroponinT', 0.3086857051131902), ('Albumin', 0.19249637129152009), ('ALP', 0.09490049574616588)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinI', 1.8952405830215657), ('Cholesterol', 1.4205580672482152), ('TroponinT', 0.5233142341164374), ('BUN', 0.22346096188397796), ('ALT', 0.16362555313936245)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.077188121476897), ('TroponinT', 0.5972769784938845), ('ALT', 0.5418725098570927), ('K', 0.15370072454147682), ('SaO2', 0.14629682982547365)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_usgan_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_usgan_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinI', 1.9335693321181284), ('Ph', 0.8942541473568749), ('TroponinT', 0.8319999392296276), ('Cholesterol', 0.47416288641362936), ('ALT', 0.25324161013363694)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Cholesterol', 2.9719090751387416), ('TroponinT', 0.9059626836070747), ('Ph', 0.8849431181997474), ('ALT', 0.6314885668513672), ('SaO2', 0.15647077724325148)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_usgan_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('Cholesterol', 2.4977461887251122), ('TroponinI', 2.02984317003595), ('ALT', 0.37824695671773023), ('ALP', 0.18814526095099215), ('BUN', 0.1542478468532586)]
----------------------------


In [ ]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[32])
print(variables[5])

Top 5 variáveis BMI
-----------------------
Cholesterol
TroponinI
ALP
TroponinT
Bilirubin


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 15, 5, 28, 2]

results_bootstrap_usgan_female = []

for i in indices:
       results_bootstrap_usgan_female.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_female,standards_deviations_usgan_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_female)

print(means_bootstraps_usgan_female)
print(standards_deviations_usgan_female)


[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]
[np.float64(0.3381506628642691), np.float64(0.08322467917877542), np.float64(0.07193090333728737), np.float64(0.02436604729618584), np.float64(0.03851788865527756)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_female, upper_bounds_usgan_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_female, means_bootstraps_usgan_female,standards_deviations_usgan_female)

print(lower_bounds_usgan_female)
print(upper_bounds_usgan_female)

[np.float64(0.5235200070760525), np.float64(0.1366787734720611), np.float64(0.3825933708508273), np.float64(0.20450627248860936), np.float64(0.3499327009241326)]
[np.float64(1.849046248200679), np.float64(0.46291352110266176), np.float64(0.6645573306838577), np.float64(0.30001942278085697), np.float64(0.5009200499738685)]


In [ ]:
lower_bounds_usgan_female, upper_bounds_usgan_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_female)

print(lower_bounds_usgan_female)
print(upper_bounds_usgan_female)

#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_usgan_female = toolkits.calc_mean_values_ci(lower_bounds_usgan_female, upper_bounds_usgan_female)

print(mean_values_ci_usgan_female)

[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6, 15, 5, 28, 2]

results_bootstrap_usgan_male = []

for i in indices:
       results_bootstrap_usgan_male.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_male,standards_deviations_usgan_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_male)

print(means_bootstraps_usgan_male)
print(standards_deviations_usgan_male)


[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]
[np.float64(0.17358454405745072), np.float64(0.025632322705534415), np.float64(0.2124414858781128), np.float64(0.03724148791663912), np.float64(0.04259881898681301)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_male, upper_bounds_usgan_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_male, means_bootstraps_usgan_male,standards_deviations_usgan_male)

print(lower_bounds_usgan_male)
print(upper_bounds_usgan_male)

[np.float64(0.5129891453019042), np.float64(0.07149955530644256), np.float64(0.2837910673267833), np.float64(0.33405772637289466), np.float64(0.4386664730964766)]
[np.float64(1.1934280545527236), np.float64(0.17197641399235403), np.float64(1.1165463896133345), np.float64(0.4800416764674872), np.float64(0.6056507750926656)]


In [ ]:
lower_bounds_usgan_male, upper_bounds_usgan_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_male)

print(lower_bounds_usgan_male)
print(upper_bounds_usgan_male)

#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_usgan_male = toolkits.calc_mean_values_ci(lower_bounds_usgan_male, upper_bounds_usgan_male)

print(mean_values_ci_usgan_male)

[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,28,5]

results_bootstrap_usgan_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_usgan_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_more_than_or_equal_to_65_subgroup,standards_deviations_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_usgan_more_than_or_equal_to_65_subgroup)
print(standards_deviations_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.9260057824625499), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]
[np.float64(0.14022222850550667), np.float64(0.2902139592129938), np.float64(0.025161049870449845), np.float64(0.08795901770065773), np.float64(0.06166376517595487)]


#### Calculating lower bound and upper bound


In [ ]:
lower_bounds_usgan_more_than_or_equal_to_65_subgroup, upper_bounds_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup, means_bootstraps_usgan_more_than_or_equal_to_65_subgroup,standards_deviations_usgan_more_than_or_equal_to_65_subgroup)

print(lower_bounds_usgan_more_than_or_equal_to_65_subgroup)
print(upper_bounds_usgan_more_than_or_equal_to_65_subgroup)


[np.float64(0.3602797122452329), np.float64(0.3571968745943058), np.float64(0.18962243507704596), np.float64(0.491014463045211), np.float64(0.45334176332908904)]
[np.float64(0.9099407476507105), np.float64(1.4948146903307942), np.float64(0.2882519381956417), np.float64(0.8358074766628315), np.float64(0.6950592811211025)]


In [ ]:
lower_bounds_usgan_more_than_or_equal_to_65_subgroup, upper_bounds_usgan_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_more_than_or_equal_to_65_subgroup)

print(lower_bounds_usgan_more_than_or_equal_to_65_subgroup)
print(upper_bounds_usgan_more_than_or_equal_to_65_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_more_than_or_equal_to_65_subgroup , upper_bounds_usgan_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_usgan_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.92600578246255), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,28,5]

results_bootstrap_usgan_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_usgan_less_than_65_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_less_than_65_subgroup,standards_deviations_usgan_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_less_than_65_subgroup)

print(means_bootstraps_usgan_less_than_65_subgroup)
print(standards_deviations_usgan_less_than_65_subgroup)


[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]
[np.float64(0.4337083267621267), np.float64(0.06965814575847884), np.float64(0.08711583404267569), np.float64(0.6101374424469065), np.float64(0.06334923482333744)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_less_than_65_subgroup, means_bootstraps_usgan_less_than_65_subgroup, standards_deviations_usgan_less_than_65_subgroup)

print(lower_bounds_usgan_less_than_65_subgroup)
print(upper_bounds_usgan_less_than_65_subgroup)

[np.float64(0.23705163755647063), np.float64(0.40032284839995347), np.float64(0.30726428110681003), np.float64(-0.41507045428487355), np.float64(0.5887783496794352)]
[np.float64(1.9371570380542658), np.float64(0.6733777622328736), np.float64(0.6487520755204355), np.float64(1.9766243713457599), np.float64(0.8371027870832592)]


In [ ]:
lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_less_than_65_subgroup)

print(lower_bounds_usgan_less_than_65_subgroup)
print(upper_bounds_usgan_less_than_65_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_less_than_65_subgroup, upper_bounds_usgan_less_than_65_subgroup)

print(mean_values_ci_usgan_less_than_65_subgroup)

[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,28,32]

results_bootstrap_usgan_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_1_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_1_subgroup, standards_deviations_usgan_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_1_subgroup)

print(means_bootstraps_usgan_ICUType_1_subgroup)
print(standards_deviations_usgan_ICUType_1_subgroup)


[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547066), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]
[np.float64(0.17812570383563214), np.float64(0.03508402734675316), np.float64(0.2710993234914266), np.float64(0.37708432885457466), np.float64(0.15436752793597142)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_1_subgroup, means_bootstraps_usgan_ICUType_1_subgroup, standards_deviations_usgan_ICUType_1_subgroup)

print(lower_bounds_usgan_ICUType_1_subgroup)
print(upper_bounds_usgan_ICUType_1_subgroup)

[np.float64(0.1735625173126043), np.float64(0.1565126112697423), np.float64(0.437748337678337), np.float64(0.6022929596856587), np.float64(0.41865583030294223)]
[np.float64(0.8718024457899787), np.float64(0.29403947133425135), np.float64(1.5004381582310762), np.float64(2.080436367064507), np.float64(1.0237654205769116)]


In [ ]:
lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_1_subgroup)

print(lower_bounds_usgan_ICUType_1_subgroup)
print(upper_bounds_usgan_ICUType_1_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_1_subgroup, upper_bounds_usgan_ICUType_1_subgroup)

print(mean_values_ci_usgan_ICUType_1_subgroup)

[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547065), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,28,32]

results_bootstrap_usgan_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_2_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_2_subgroup, standards_deviations_usgan_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_2_subgroup)

print(means_bootstraps_usgan_ICUType_2_subgroup)
print(standards_deviations_usgan_ICUType_2_subgroup)


[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.3769132654783525), np.float64(0.17130476566918515)]
[np.float64(0.24746020204511532), np.float64(0.02844864967364565), np.float64(0.0), np.float64(0.12241202033633213), np.float64(0.04748266503243493)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_2_subgroup, means_bootstraps_usgan_ICUType_2_subgroup, standards_deviations_usgan_ICUType_2_subgroup)

print(lower_bounds_usgan_ICUType_2_subgroup)
print(upper_bounds_usgan_ICUType_2_subgroup)

NameError: name 'toolkits' is not defined

In [ ]:
lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_2_subgroup)

print(lower_bounds_usgan_ICUType_2_subgroup)
print(upper_bounds_usgan_ICUType_2_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_2_subgroup, upper_bounds_usgan_ICUType_2_subgroup)

print(mean_values_ci_usgan_ICUType_2_subgroup)

[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.37691326547835247), np.float64(0.17130476566918512)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,28,32]

results_bootstrap_usgan_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_3_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_3_subgroup, standards_deviations_usgan_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_3_subgroup)

print(means_bootstraps_usgan_ICUType_3_subgroup)
print(standards_deviations_usgan_ICUType_3_subgroup)


[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]
[np.float64(0.4559261164801034), np.float64(0.45097433675337983), np.float64(0.1878586172758027), np.float64(0.06133680568713781), np.float64(0.03548543506753726)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_3_subgroup, means_bootstraps_usgan_ICUType_3_subgroup, standards_deviations_usgan_ICUType_3_subgroup)

print(lower_bounds_usgan_ICUType_3_subgroup)
print(upper_bounds_usgan_ICUType_3_subgroup)

[np.float64(0.6996377293253487), np.float64(0.25263761572853904), np.float64(0.3276231392998342), np.float64(0.4209399971546227), np.float64(0.16790862349933133)]
[np.float64(2.4868352651497814), np.float64(2.020424531705464), np.float64(1.0640153873919687), np.float64(0.661375857301666), np.float64(0.30700897291554674)]


In [ ]:
lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_3_subgroup)

print(lower_bounds_usgan_ICUType_3_subgroup)
print(upper_bounds_usgan_ICUType_3_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_3_subgroup, upper_bounds_usgan_ICUType_3_subgroup)

print(mean_values_ci_usgan_ICUType_3_subgroup)

[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,28,32]

results_bootstrap_usgan_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_usgan_ICUType_4_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_ICUType_4_subgroup, standards_deviations_usgan_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_ICUType_4_subgroup)

print(means_bootstraps_usgan_ICUType_4_subgroup)
print(standards_deviations_usgan_ICUType_4_subgroup)


[np.float64(0.44306473111336137), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]
[np.float64(0.0935566892280583), np.float64(0.08074364451634707), np.float64(0.2681495472441715), np.float64(0.2724504216824265), np.float64(0.022189091660584512)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_ICUType_4_subgroup, means_bootstraps_usgan_ICUType_4_subgroup, standards_deviations_usgan_ICUType_4_subgroup)

print(lower_bounds_usgan_ICUType_4_subgroup)
print(upper_bounds_usgan_ICUType_4_subgroup)

[np.float64(0.25969698971356064), np.float64(0.33294655262192163), np.float64(0.2031662976673827), np.float64(0.1584331675516304), np.float64(0.13677304273715427)]
[np.float64(0.626432472513162), np.float64(0.6494558230870122), np.float64(1.2542932078059783), np.float64(1.2264191956922437), np.float64(0.22375268374596166)]


In [ ]:
lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_ICUType_4_subgroup)

print(lower_bounds_usgan_ICUType_4_subgroup)
print(upper_bounds_usgan_ICUType_4_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_ICUType_4_subgroup, upper_bounds_usgan_ICUType_4_subgroup)

print(mean_values_ci_usgan_ICUType_4_subgroup)

[np.float64(0.4430647311133613), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_undefined_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_undefined_classification_subgroup, standards_deviations_usgan_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_undefined_classification_subgroup)

print(means_bootstraps_usgan_undefined_classification_subgroup)
print(standards_deviations_usgan_undefined_classification_subgroup)


[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]
[np.float64(0.05986934359644469), np.float64(0.17840974136243826), np.float64(0.044640773548291844), np.float64(0.2853216956527157), np.float64(0.12549413739177268)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_undefined_classification_subgroup, means_bootstraps_usgan_undefined_classification_subgroup, standards_deviations_usgan_undefined_classification_subgroup)

print(lower_bounds_usgan_undefined_classification_subgroup)
print(upper_bounds_usgan_undefined_classification_subgroup)

[np.float64(0.28951948015518403), np.float64(0.22394384394790923), np.float64(0.18842469528371564), np.float64(0.28379016616066055), np.float64(0.16723269590166293)]
[np.float64(0.5242029946093546), np.float64(0.9232971790708792), np.float64(0.3634133120770363), np.float64(1.4022306611351032), np.float64(0.6591606750192545)]


In [ ]:
lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_undefined_classification_subgroup)

print(lower_bounds_usgan_undefined_classification_subgroup)
print(upper_bounds_usgan_undefined_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_undefined_classification_subgroup, upper_bounds_usgan_undefined_classification_subgroup)

print(mean_values_ci_usgan_undefined_classification_subgroup)

[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_low_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_low_weight_classification_subgroup, standards_deviations_usgan_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_low_weight_classification_subgroup)

print(means_bootstraps_usgan_low_weight_classification_subgroup)
print(standards_deviations_usgan_low_weight_classification_subgroup)


[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032636), np.float64(0.0), np.float64(1.4606367473586466)]
[np.float64(1.666676645356208), np.float64(0.05309602591127397), np.float64(0.3139178166078663), np.float64(0.0), np.float64(1.2916342659860158)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_low_weight_classification_subgroup, means_bootstraps_usgan_low_weight_classification_subgroup, standards_deviations_usgan_low_weight_classification_subgroup)

print(lower_bounds_usgan_low_weight_classification_subgroup)
print(upper_bounds_usgan_low_weight_classification_subgroup)

[np.float64(-1.2543455052071963), np.float64(-0.03473297672088424), np.float64(-0.2048614788536039), np.float64(0.0), np.float64(-1.070919895171773)]
[np.float64(5.278906892337211), np.float64(0.17339962029572073), np.float64(1.0256737504601312), np.float64(0.0), np.float64(3.9921933898890662)]


In [ ]:
lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_low_weight_classification_subgroup)

print(lower_bounds_usgan_low_weight_classification_subgroup)
print(upper_bounds_usgan_low_weight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_low_weight_classification_subgroup, upper_bounds_usgan_low_weight_classification_subgroup)

print(mean_values_ci_usgan_low_weight_classification_subgroup)

[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032637), np.float64(0.0), np.float64(1.4606367473586466)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_normal_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_normal_weight_classification_subgroup, standards_deviations_usgan_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_normal_weight_classification_subgroup)

print(means_bootstraps_usgan_normal_weight_classification_subgroup)
print(standards_deviations_usgan_normal_weight_classification_subgroup)


[np.float64(0.548686639105505), np.float64(0.7625897872503594), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]
[np.float64(0.12623025707018218), np.float64(0.3731308327837512), np.float64(0.07341434562099826), np.float64(0.089947135630567), np.float64(0.2013724566774524)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_normal_weight_classification_subgroup, means_bootstraps_usgan_normal_weight_classification_subgroup, standards_deviations_usgan_normal_weight_classification_subgroup)

print(lower_bounds_usgan_normal_weight_classification_subgroup)
print(upper_bounds_usgan_normal_weight_classification_subgroup)

[np.float64(0.3012798814887154), np.float64(0.03126679347276984), np.float64(0.10543028800815332), np.float64(0.38859566088565467), np.float64(0.12860300267586205)]
[np.float64(0.7960933967222945), np.float64(1.4939127810279489), np.float64(0.3932092347396181), np.float64(0.7411819535825561), np.float64(0.9179685278081802)]


In [ ]:
lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_normal_weight_classification_subgroup)

print(lower_bounds_usgan_normal_weight_classification_subgroup)
print(upper_bounds_usgan_normal_weight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_normal_weight_classification_subgroup, upper_bounds_usgan_normal_weight_classification_subgroup)

print(mean_values_ci_usgan_normal_weight_classification_subgroup)

[np.float64(0.548686639105505), np.float64(0.7625897872503593), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_overweight_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_overweight_classification_subgroup, standards_deviations_usgan_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_overweight_classification_subgroup)

print(means_bootstraps_usgan_overweight_classification_subgroup)
print(standards_deviations_usgan_overweight_classification_subgroup)


[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082165), np.float64(0.1969866594273391)]
[np.float64(0.05690905629749502), np.float64(0.2748051570149171), np.float64(0.190452089816047), np.float64(0.5659200713279424), np.float64(0.06465085722820384)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_overweight_classification_subgroup, means_bootstraps_usgan_overweight_classification_subgroup, standards_deviations_usgan_overweight_classification_subgroup)

print(lower_bounds_usgan_overweight_classification_subgroup)
print(upper_bounds_usgan_overweight_classification_subgroup)

[np.float64(0.16821060322753356), np.float64(0.9483102156492423), np.float64(0.07116983979697916), np.float64(-0.06774702542288913), np.float64(0.07027330769041853)]
[np.float64(0.3912900047020388), np.float64(2.0255266366794666), np.float64(0.8177283134366586), np.float64(2.150618890439322), np.float64(0.3237000111642596)]


In [ ]:
lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_overweight_classification_subgroup)

print(lower_bounds_usgan_overweight_classification_subgroup)
print(upper_bounds_usgan_overweight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_overweight_classification_subgroup, upper_bounds_usgan_overweight_classification_subgroup)

print(mean_values_ci_usgan_overweight_classification_subgroup)

[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082163), np.float64(0.19698665942733906)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs1_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_obs1_classification_subgroup, standards_deviations_usgan_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs1_classification_subgroup)

print(means_bootstraps_usgan_obs1_classification_subgroup)
print(standards_deviations_usgan_obs1_classification_subgroup)


[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]
[np.float64(0.05798396274724235), np.float64(0.25089896187847854), np.float64(0.08655489332963262), np.float64(0.19483046174517651), np.float64(0.12094499608671105)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs1_classification_subgroup, means_bootstraps_usgan_obs1_classification_subgroup, standards_deviations_usgan_obs1_classification_subgroup)

print(lower_bounds_usgan_obs1_classification_subgroup)
print(upper_bounds_usgan_obs1_classification_subgroup)

[np.float64(0.19364783590608406), np.float64(0.0070376496942498035), np.float64(0.10522008285767387), np.float64(0.11111241585757914), np.float64(0.1681406336979566)]
[np.float64(0.42094079323709843), np.float64(0.9905435077748617), np.float64(0.44450903008124604), np.float64(0.8748337920812886), np.float64(0.6422363065785395)]


In [ ]:
lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs1_classification_subgroup)

print(lower_bounds_usgan_obs1_classification_subgroup)
print(upper_bounds_usgan_obs1_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs1_classification_subgroup, upper_bounds_usgan_obs1_classification_subgroup)

print(mean_values_ci_usgan_obs1_classification_subgroup)

[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs2_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_obs2_classification_subgroup, standards_deviations_usgan_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs2_classification_subgroup)

print(means_bootstraps_usgan_obs2_classification_subgroup)
print(standards_deviations_usgan_obs2_classification_subgroup)


[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.3961240813101228), np.float64(0.0), np.float64(0.2858736875756382)]
[np.float64(0.049002577704985846), np.float64(0.09598273849535295), np.float64(1.0268585416236724), np.float64(0.0), np.float64(0.13790114187609237)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs2_classification_subgroup, means_bootstraps_usgan_obs2_classification_subgroup, standards_deviations_usgan_obs2_classification_subgroup)

print(lower_bounds_usgan_obs2_classification_subgroup)
print(upper_bounds_usgan_obs2_classification_subgroup)

[np.float64(0.1605865061227179), np.float64(-0.062157550347901436), np.float64(-0.6164816774895991), np.float64(0.0), np.float64(0.015592416071548876)]
[np.float64(0.35267308102551326), np.float64(0.3140878708289346), np.float64(3.408729840109845), np.float64(0.0), np.float64(0.5561549590797275)]


In [ ]:
lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_usgan_obs2_classification_subgroup)

print(lower_bounds_usgan_obs2_classification_subgroup)
print(upper_bounds_usgan_obs2_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs2_classification_subgroup, upper_bounds_usgan_obs2_classification_subgroup)

print(mean_values_ci_usgan_obs2_classification_subgroup)

[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.396124081310123), np.float64(0.0), np.float64(0.28587368757563814)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,31,0,32,5]

results_bootstrap_usgan_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_usgan_obs3_classification_subgroup.append(toolkits.bootstrap(testing_ae_usgan_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_usgan_obs3_classification_subgroup)

print(means_bootstraps_usgan_obs3_classification_subgroup)
print(standards_deviations_usgan_obs3_classification_subgroup)


[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.20886002502599546)]
[np.float64(0.050281311239894225), np.float64(0.0), np.float64(0.8237478596037178), np.float64(0.19004214803335054), np.float64(0.12978497928121874)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs3_classification_subgroup, means_bootstraps_usgan_obs3_classification_subgroup, standards_deviations_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

[np.float64(0.05300461518183637), np.float64(0.0), np.float64(-0.4551198683898825), np.float64(-0.12823364717905072), np.float64(-0.045513860099470366)]
[np.float64(0.25010373343311976), np.float64(0.0), np.float64(2.773912405940605), np.float64(0.6167178842009423), np.float64(0.4632339101514613)]


In [ ]:
lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_usgan_obs3_classification_subgroup)

print(lower_bounds_usgan_obs3_classification_subgroup)
print(upper_bounds_usgan_obs3_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_usgan_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_usgan_obs3_classification_subgroup, upper_bounds_usgan_obs3_classification_subgroup)

print(mean_values_ci_usgan_obs3_classification_subgroup)

[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.2088600250259955)]


## GPVAE

### Calculate MAE and AE

In [ ]:
testing_mae_gpvae_variables_standard, testing_ae_gpvae_variables_standard = toolkits.calculate_mae(gpvae_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for GPVAE model

#### Difference between gender group

In [ ]:
diff_gpvae_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 1, 2, variables)
diff_gpvae_gender_subgroup

[('Cholesterol', 0.7169863050614339),
 ('SaO2', 0.20890310598463563),
 ('AST', 0.20239902682582445),
 ('Creatinine', 0.14547286040991392),
 ('Lactate', 0.10004376501079093)]

#### Difference between age group

In [ ]:
diff_gpvae_age_subgroup = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 4, 5, variables)
diff_gpvae_age_subgroup

[('Cholesterol', 0.49957543635081736),
 ('ALP', 0.44452750749784475),
 ('TroponinI', 0.2618898374982579),
 ('SaO2', 0.21620679352892658),
 ('Albumin', 0.1910974661437378)]

#### Difference between ICUType group

In [ ]:
diff_gpvae_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 7, variables)
diff_gpvae_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 8, variables)
diff_gpvae_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 6, 9, variables)
diff_gpvae_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 7, 8, variables)
diff_gpvae_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 7, 9, variables)
diff_gpvae_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 8, 9, variables)

In [ ]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_gpvae_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_gpvae_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_gpvae_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_gpvae_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.9502364805777341), ('TroponinI', 0.8918236664146777), ('TroponinT', 0.3114883755971654), ('HCO3', 0.23628939822722694), ('SaO2', 0.23145972909185003)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 0.9937353873628099), ('ALP', 0.8302413758449083), ('TroponinI', 0.7412819088709723), ('Cholesterol', 0.406672113721821), ('TroponinT', 0.37481609205650046)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinT', 0.4347207972367254), ('TroponinI', 0.4317269923877344), ('Cholesterol', 0.31119956124464043), ('ALP', 0.24020355084081502), ('Bilirubin', 0.2227724629487818)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.22519511645466), ('ALP', 0.7677499234797236), ('Cholesterol', 0.5435643668559131), ('PaCO2', 0.40728844810871434), ('HCO3', 0.3961385611965824)]
----------------------------
Diferença(MAE

In [ ]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[31])
print(variables[0])
print(variables[6])
print(variables[32])

Top 5 variáveis ICUType
-----------------------
SaO2
ALP
Cholesterol
TroponinI
TroponinT


#### Difference between BMI group

In [ ]:
diff_gpvae_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 11, variables)
diff_gpvae_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 12, variables)
diff_gpvae_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 13, variables)
diff_gpvae_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 14, variables)
diff_gpvae_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 15, variables)
diff_gpvae_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 10, 16, variables)

diff_gpvae_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 12, variables)
diff_gpvae_classification_low_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 13, variables)
diff_gpvae_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 14, variables)
diff_gpvae_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 15, variables)
diff_gpvae_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 11, 16, variables)

diff_gpvae_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 13, variables)
diff_gpvae_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 14, variables)
diff_gpvae_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 15, variables)
diff_gpvae_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 12, 16, variables)

diff_gpvae_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 14, variables)
diff_gpvae_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 15, variables)
diff_gpvae_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 13, 16, variables)


diff_gpvae_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 14, 15, variables)
diff_gpvae_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 14, 16, variables)

diff_gpvae_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_gpvae_variables_standard, 15, 16, variables)

In [ ]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_gpvae_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_gpvae_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_gpvae_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_gpvae_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_gpvae_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_gpvae_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.4984287714039075), ('Bilirubin', 0.9811191215037678), ('TroponinI', 0.8126922038658223), ('Cholesterol', 0.5201117490423033), ('Lactate', 0.4554101566283363)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('PaCO2', 0.26413489264298506), ('SaO2', 0.2556269283251386), ('ALT', 0.20532099691202862), ('Platelets', 0.1705636160025804), ('HCO3', 0.1633457333551286)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.8097408650488799), ('TroponinI', 0.3305148845346736), ('TroponinT', 0.2897346238466169), ('PaCO2', 0.2579367576470314), ('SaO2', 0.24006883512626825)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8803621039317031), ('SaO2', 0.2591527877822734), ('PaCO2', 0.2568282809114367), ('Cholesterol', 0.23640482047021322), ('H

In [ ]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_gpvae_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_gpvae_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_gpvae_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_gpvae_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_gpvae_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.403021537470746), ('Bilirubin', 0.9461369295522424), ('TroponinI', 0.6835013443205654), ('Cholesterol', 0.5801292127536045), ('HCO3', 0.5142557195063615)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.5835896949503965), ('Cholesterol', 1.329852614091183), ('TroponinI', 1.143207088400496), ('Bilirubin', 1.128743649214257), ('HCO3', 0.5693350874264985)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.6272017145736122), ('Bilirubin', 1.0852515823152353), ('Ph', 0.8728858021745433), ('TroponinI', 0.5900021366441112), ('HCO3', 0.581347050390737)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.6404632350073978), ('Bilirubin', 1.0958944421751695), ('TroponinT', 0.7259970588377027), ('HCO3', 0.5949372708589092), ('ALT', 0.49226456329646173)]
-----

In [ ]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_gpvae_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_gpvae_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_gpvae_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_gpvae_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('Cholesterol', 0.7497234013375786), ('TroponinI', 0.4597057440799305), ('TroponinT', 0.35357540568797513), ('ALT', 0.32170038759579944), ('Bilirubin', 0.18260671966201464)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8880086026672669), ('Cholesterol', 0.29642228418151445), ('ALP', 0.22418017710286614), ('Mg', 0.17276537650943447), ('Creatinine', 0.16845587072682183)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 1.01714784202371), ('TroponinI', 0.6835013443205654), ('ALT', 0.539823955147861), ('Cholesterol', 0.33531144860598827), ('BUN', 0.2971806787390593)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('ALT', 1.435649509729886), ('TroponinT', 0.8074637349193878), ('Cholesterol', 0.6091621498209887), ('TroponinI', 0.41584146021681445), ('W

In [ ]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_gpvae_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_gpvae_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_gpvae_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Cholesterol', 1.0461456855190931), ('Ph', 0.8934708178133868), ('TroponinI', 0.5532049517563847), ('TroponinT', 0.3830546982731618), ('ALT', 0.2675466002043284)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('TroponinI', 1.143207088400496), ('Cholesterol', 1.085034849943567), ('TroponinT', 0.6635724363357348), ('BUN', 0.3170724188083759), ('AST', 0.21918031375498315)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.3588855511585674), ('ALT', 1.1139491221340865), ('TroponinI', 0.8755472042967449), ('TroponinT', 0.45388832923141276), ('Weight', 0.3685338792048445)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_gpvae_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_gpvae_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.0466271346088967), ('Ph', 0.8973569138255766), ('TroponinI', 0.5900021366441112), ('ALT', 0.48567016775638994), ('BUN', 0.25576191544464466)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('ALT', 1.381495722338415), ('Ph', 0.8941617584582486), ('TroponinT', 0.8369430275045746), ('Glucose', 0.3283979292549837), ('TroponinI', 0.3223422525403602)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_gpvae_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('ALT', 0.895825554582025), ('HCO3', 0.3183493427852112), ('Cholesterol', 0.27385070121500044), ('TroponinI', 0.26765988410375097), ('Weight', 0.26357190666548425)]
----------------------------


In [ ]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[1])
print(variables[31])
print(variables[6])
print(variables[0])
print(variables[32])

Top 5 variáveis BMI
-----------------------
ALT
TroponinI
Cholesterol
ALP
TroponinT


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,28,2,7,16]

results_bootstrap_gpvae_female = []

for i in indices:
       results_bootstrap_gpvae_female.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_female,standards_deviations_gpvae_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_female)

print(means_bootstraps_gpvae_female)
print(standards_deviations_gpvae_female)


[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]
[np.float64(0.3381506628642691), np.float64(0.08322467917877542), np.float64(0.07193090333728737), np.float64(0.02436604729618584), np.float64(0.03851788865527756)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_female, upper_bounds_gpvae_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_female, means_bootstraps_gpvae_female,standards_deviations_gpvae_female)

print(lower_bounds_gpvae_female)
print(upper_bounds_gpvae_female)

[np.float64(0.5235200070760525), np.float64(0.1366787734720611), np.float64(0.3825933708508273), np.float64(0.20450627248860936), np.float64(0.3499327009241326)]
[np.float64(1.849046248200679), np.float64(0.46291352110266176), np.float64(0.6645573306838577), np.float64(0.30001942278085697), np.float64(0.5009200499738685)]


In [ ]:
lower_bounds_gpvae_female, upper_bounds_gpvae_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_female)

print(lower_bounds_gpvae_female)
print(upper_bounds_gpvae_female)

#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_gpvae_female = toolkits.calc_mean_values_ci(lower_bounds_gpvae_female, upper_bounds_gpvae_female)

print(mean_values_ci_gpvae_female)

[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,28,2,7,16]

results_bootstrap_gpvae_male = []

for i in indices:
       results_bootstrap_gpvae_male.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_male,standards_deviations_gpvae_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_male)

print(means_bootstraps_gpvae_male)
print(standards_deviations_gpvae_male)


[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]
[np.float64(0.17358454405745072), np.float64(0.025632322705534415), np.float64(0.2124414858781128), np.float64(0.03724148791663912), np.float64(0.04259881898681301)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_male, upper_bounds_gpvae_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_male, means_bootstraps_gpvae_male,standards_deviations_gpvae_male)

print(lower_bounds_gpvae_male)
print(upper_bounds_gpvae_male)

[np.float64(0.5129891453019042), np.float64(0.07149955530644256), np.float64(0.2837910673267833), np.float64(0.33405772637289466), np.float64(0.4386664730964766)]
[np.float64(1.1934280545527236), np.float64(0.17197641399235403), np.float64(1.1165463896133345), np.float64(0.4800416764674872), np.float64(0.6056507750926656)]


In [ ]:
lower_bounds_gpvae_male, upper_bounds_gpvae_male = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_male)

print(lower_bounds_gpvae_male)
print(upper_bounds_gpvae_male)

#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_gpvae_male = toolkits.calc_mean_values_ci(lower_bounds_gpvae_male, upper_bounds_gpvae_male)

print(mean_values_ci_gpvae_male)

[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,28,3]

results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup,standards_deviations_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup)
print(standards_deviations_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.9260057824625499), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]
[np.float64(0.14022222850550667), np.float64(0.2902139592129938), np.float64(0.025161049870449845), np.float64(0.08795901770065773), np.float64(0.06166376517595487)]


#### Calculating lower bound and upper bound


In [ ]:
lower_bounds_gpvae_more_than_or_equal_to_65_subgroup, upper_bounds_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup, means_bootstraps_gpvae_more_than_or_equal_to_65_subgroup,standards_deviations_gpvae_more_than_or_equal_to_65_subgroup)

print(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup)
print(upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)


[np.float64(0.3602797122452329), np.float64(0.3571968745943058), np.float64(0.18962243507704596), np.float64(0.491014463045211), np.float64(0.45334176332908904)]
[np.float64(0.9099407476507105), np.float64(1.4948146903307942), np.float64(0.2882519381956417), np.float64(0.8358074766628315), np.float64(0.6950592811211025)]


In [ ]:
lower_bounds_gpvae_more_than_or_equal_to_65_subgroup, upper_bounds_gpvae_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_more_than_or_equal_to_65_subgroup)

print(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup)
print(upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_more_than_or_equal_to_65_subgroup , upper_bounds_gpvae_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_gpvae_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.92600578246255), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,28,3]

results_bootstrap_gpvae_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_gpvae_less_than_65_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_less_than_65_subgroup,standards_deviations_gpvae_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_less_than_65_subgroup)

print(means_bootstraps_gpvae_less_than_65_subgroup)
print(standards_deviations_gpvae_less_than_65_subgroup)


[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]
[np.float64(0.4337083267621267), np.float64(0.06965814575847884), np.float64(0.08711583404267569), np.float64(0.6101374424469065), np.float64(0.06334923482333744)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_less_than_65_subgroup, means_bootstraps_gpvae_less_than_65_subgroup, standards_deviations_gpvae_less_than_65_subgroup)

print(lower_bounds_gpvae_less_than_65_subgroup)
print(upper_bounds_gpvae_less_than_65_subgroup)

[np.float64(0.23705163755647063), np.float64(0.40032284839995347), np.float64(0.30726428110681003), np.float64(-0.41507045428487355), np.float64(0.5887783496794352)]
[np.float64(1.9371570380542658), np.float64(0.6733777622328736), np.float64(0.6487520755204355), np.float64(1.9766243713457599), np.float64(0.8371027870832592)]


In [ ]:
lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_less_than_65_subgroup)

print(lower_bounds_gpvae_less_than_65_subgroup)
print(upper_bounds_gpvae_less_than_65_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_less_than_65_subgroup, upper_bounds_gpvae_less_than_65_subgroup)

print(mean_values_ci_gpvae_less_than_65_subgroup)

[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,31,0,6,32]

results_bootstrap_gpvae_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_1_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_1_subgroup, standards_deviations_gpvae_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_1_subgroup)

print(means_bootstraps_gpvae_ICUType_1_subgroup)
print(standards_deviations_gpvae_ICUType_1_subgroup)


[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547066), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]
[np.float64(0.17812570383563214), np.float64(0.03508402734675316), np.float64(0.2710993234914266), np.float64(0.37708432885457466), np.float64(0.15436752793597142)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_1_subgroup, means_bootstraps_gpvae_ICUType_1_subgroup, standards_deviations_gpvae_ICUType_1_subgroup)

print(lower_bounds_gpvae_ICUType_1_subgroup)
print(upper_bounds_gpvae_ICUType_1_subgroup)

[np.float64(0.1735625173126043), np.float64(0.1565126112697423), np.float64(0.437748337678337), np.float64(0.6022929596856587), np.float64(0.41865583030294223)]
[np.float64(0.8718024457899787), np.float64(0.29403947133425135), np.float64(1.5004381582310762), np.float64(2.080436367064507), np.float64(1.0237654205769116)]


In [ ]:
lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_1_subgroup)

print(lower_bounds_gpvae_ICUType_1_subgroup)
print(upper_bounds_gpvae_ICUType_1_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_1_subgroup, upper_bounds_gpvae_ICUType_1_subgroup)

print(mean_values_ci_gpvae_ICUType_1_subgroup)

[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547065), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,31,0,6,32]

results_bootstrap_gpvae_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_2_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_2_subgroup, standards_deviations_gpvae_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_2_subgroup)

print(means_bootstraps_gpvae_ICUType_2_subgroup)
print(standards_deviations_gpvae_ICUType_2_subgroup) 


[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.3769132654783525), np.float64(0.17130476566918515)]
[np.float64(0.24746020204511532), np.float64(0.02844864967364565), np.float64(0.0), np.float64(0.12241202033633213), np.float64(0.04748266503243493)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_2_subgroup, means_bootstraps_gpvae_ICUType_2_subgroup, standards_deviations_gpvae_ICUType_2_subgroup)

print(lower_bounds_gpvae_ICUType_2_subgroup)
print(upper_bounds_gpvae_ICUType_2_subgroup)

NameError: name 'toolkits' is not defined

In [ ]:
lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_2_subgroup)

print(lower_bounds_gpvae_ICUType_2_subgroup)
print(upper_bounds_gpvae_ICUType_2_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_2_subgroup, upper_bounds_gpvae_ICUType_2_subgroup)

print(mean_values_ci_gpvae_ICUType_2_subgroup)

[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.37691326547835247), np.float64(0.17130476566918512)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,31,0,6,32]

results_bootstrap_gpvae_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_3_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_3_subgroup, standards_deviations_gpvae_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_3_subgroup)

print(means_bootstraps_gpvae_ICUType_3_subgroup)
print(standards_deviations_gpvae_ICUType_3_subgroup)


[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]
[np.float64(0.4559261164801034), np.float64(0.45097433675337983), np.float64(0.1878586172758027), np.float64(0.06133680568713781), np.float64(0.03548543506753726)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_3_subgroup, means_bootstraps_gpvae_ICUType_3_subgroup, standards_deviations_gpvae_ICUType_3_subgroup)

print(lower_bounds_gpvae_ICUType_3_subgroup)
print(upper_bounds_gpvae_ICUType_3_subgroup)

[np.float64(0.6996377293253487), np.float64(0.25263761572853904), np.float64(0.3276231392998342), np.float64(0.4209399971546227), np.float64(0.16790862349933133)]
[np.float64(2.4868352651497814), np.float64(2.020424531705464), np.float64(1.0640153873919687), np.float64(0.661375857301666), np.float64(0.30700897291554674)]


In [ ]:
lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_3_subgroup)

print(lower_bounds_gpvae_ICUType_3_subgroup)
print(upper_bounds_gpvae_ICUType_3_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_3_subgroup, upper_bounds_gpvae_ICUType_3_subgroup)

print(mean_values_ci_gpvae_ICUType_3_subgroup)

[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,31,0,6,32]

results_bootstrap_gpvae_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_gpvae_ICUType_4_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_ICUType_4_subgroup, standards_deviations_gpvae_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_ICUType_4_subgroup)

print(means_bootstraps_gpvae_ICUType_4_subgroup)
print(standards_deviations_gpvae_ICUType_4_subgroup)


[np.float64(0.44306473111336137), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]
[np.float64(0.0935566892280583), np.float64(0.08074364451634707), np.float64(0.2681495472441715), np.float64(0.2724504216824265), np.float64(0.022189091660584512)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_ICUType_4_subgroup, means_bootstraps_gpvae_ICUType_4_subgroup, standards_deviations_gpvae_ICUType_4_subgroup)

print(lower_bounds_gpvae_ICUType_4_subgroup)
print(upper_bounds_gpvae_ICUType_4_subgroup)

[np.float64(0.25969698971356064), np.float64(0.33294655262192163), np.float64(0.2031662976673827), np.float64(0.1584331675516304), np.float64(0.13677304273715427)]
[np.float64(0.626432472513162), np.float64(0.6494558230870122), np.float64(1.2542932078059783), np.float64(1.2264191956922437), np.float64(0.22375268374596166)]


In [ ]:
lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_ICUType_4_subgroup)

print(lower_bounds_gpvae_ICUType_4_subgroup)
print(upper_bounds_gpvae_ICUType_4_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_ICUType_4_subgroup, upper_bounds_gpvae_ICUType_4_subgroup)

print(mean_values_ci_gpvae_ICUType_4_subgroup)

[np.float64(0.4430647311133613), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_undefined_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_undefined_classification_subgroup, standards_deviations_gpvae_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_undefined_classification_subgroup)

print(means_bootstraps_gpvae_undefined_classification_subgroup)
print(standards_deviations_gpvae_undefined_classification_subgroup)


[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]
[np.float64(0.05986934359644469), np.float64(0.17840974136243826), np.float64(0.044640773548291844), np.float64(0.2853216956527157), np.float64(0.12549413739177268)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_undefined_classification_subgroup, means_bootstraps_gpvae_undefined_classification_subgroup, standards_deviations_gpvae_undefined_classification_subgroup)

print(lower_bounds_gpvae_undefined_classification_subgroup)
print(upper_bounds_gpvae_undefined_classification_subgroup)

[np.float64(0.28951948015518403), np.float64(0.22394384394790923), np.float64(0.18842469528371564), np.float64(0.28379016616066055), np.float64(0.16723269590166293)]
[np.float64(0.5242029946093546), np.float64(0.9232971790708792), np.float64(0.3634133120770363), np.float64(1.4022306611351032), np.float64(0.6591606750192545)]


In [ ]:
lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_undefined_classification_subgroup)

print(lower_bounds_gpvae_undefined_classification_subgroup)
print(upper_bounds_gpvae_undefined_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_undefined_classification_subgroup, upper_bounds_gpvae_undefined_classification_subgroup)

print(mean_values_ci_gpvae_undefined_classification_subgroup)

[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_low_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_low_weight_classification_subgroup, standards_deviations_gpvae_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_low_weight_classification_subgroup)

print(means_bootstraps_gpvae_low_weight_classification_subgroup)
print(standards_deviations_gpvae_low_weight_classification_subgroup)


[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032636), np.float64(0.0), np.float64(1.4606367473586466)]
[np.float64(1.666676645356208), np.float64(0.05309602591127397), np.float64(0.3139178166078663), np.float64(0.0), np.float64(1.2916342659860158)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_low_weight_classification_subgroup, means_bootstraps_gpvae_low_weight_classification_subgroup, standards_deviations_gpvae_low_weight_classification_subgroup)

print(lower_bounds_gpvae_low_weight_classification_subgroup)
print(upper_bounds_gpvae_low_weight_classification_subgroup)

[np.float64(-1.2543455052071963), np.float64(-0.03473297672088424), np.float64(-0.2048614788536039), np.float64(0.0), np.float64(-1.070919895171773)]
[np.float64(5.278906892337211), np.float64(0.17339962029572073), np.float64(1.0256737504601312), np.float64(0.0), np.float64(3.9921933898890662)]


In [ ]:
lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_low_weight_classification_subgroup)

print(lower_bounds_gpvae_low_weight_classification_subgroup)
print(upper_bounds_gpvae_low_weight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_low_weight_classification_subgroup, upper_bounds_gpvae_low_weight_classification_subgroup)

print(mean_values_ci_gpvae_low_weight_classification_subgroup)

[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032637), np.float64(0.0), np.float64(1.4606367473586466)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_normal_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_normal_weight_classification_subgroup, standards_deviations_gpvae_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_normal_weight_classification_subgroup)

print(means_bootstraps_gpvae_normal_weight_classification_subgroup)
print(standards_deviations_gpvae_normal_weight_classification_subgroup)


[np.float64(0.548686639105505), np.float64(0.7625897872503594), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]
[np.float64(0.12623025707018218), np.float64(0.3731308327837512), np.float64(0.07341434562099826), np.float64(0.089947135630567), np.float64(0.2013724566774524)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_normal_weight_classification_subgroup, means_bootstraps_gpvae_normal_weight_classification_subgroup, standards_deviations_gpvae_normal_weight_classification_subgroup)

print(lower_bounds_gpvae_normal_weight_classification_subgroup)
print(upper_bounds_gpvae_normal_weight_classification_subgroup)

[np.float64(0.3012798814887154), np.float64(0.03126679347276984), np.float64(0.10543028800815332), np.float64(0.38859566088565467), np.float64(0.12860300267586205)]
[np.float64(0.7960933967222945), np.float64(1.4939127810279489), np.float64(0.3932092347396181), np.float64(0.7411819535825561), np.float64(0.9179685278081802)]


In [ ]:
lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_normal_weight_classification_subgroup)

print(lower_bounds_gpvae_normal_weight_classification_subgroup)
print(upper_bounds_gpvae_normal_weight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_normal_weight_classification_subgroup, upper_bounds_gpvae_normal_weight_classification_subgroup)

print(mean_values_ci_gpvae_normal_weight_classification_subgroup)

[np.float64(0.548686639105505), np.float64(0.7625897872503593), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_overweight_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_overweight_classification_subgroup, standards_deviations_gpvae_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_overweight_classification_subgroup)

print(means_bootstraps_gpvae_overweight_classification_subgroup)
print(standards_deviations_gpvae_overweight_classification_subgroup)


[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082165), np.float64(0.1969866594273391)]
[np.float64(0.05690905629749502), np.float64(0.2748051570149171), np.float64(0.190452089816047), np.float64(0.5659200713279424), np.float64(0.06465085722820384)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_overweight_classification_subgroup, means_bootstraps_gpvae_overweight_classification_subgroup, standards_deviations_gpvae_overweight_classification_subgroup)

print(lower_bounds_gpvae_overweight_classification_subgroup)
print(upper_bounds_gpvae_overweight_classification_subgroup)

[np.float64(0.16821060322753356), np.float64(0.9483102156492423), np.float64(0.07116983979697916), np.float64(-0.06774702542288913), np.float64(0.07027330769041853)]
[np.float64(0.3912900047020388), np.float64(2.0255266366794666), np.float64(0.8177283134366586), np.float64(2.150618890439322), np.float64(0.3237000111642596)]


In [ ]:
lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_overweight_classification_subgroup)

print(lower_bounds_gpvae_overweight_classification_subgroup)
print(upper_bounds_gpvae_overweight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_overweight_classification_subgroup, upper_bounds_gpvae_overweight_classification_subgroup)

print(mean_values_ci_gpvae_overweight_classification_subgroup)

[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082163), np.float64(0.19698665942733906)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs1_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_obs1_classification_subgroup, standards_deviations_gpvae_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs1_classification_subgroup)

print(means_bootstraps_gpvae_obs1_classification_subgroup)
print(standards_deviations_gpvae_obs1_classification_subgroup)


[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]
[np.float64(0.05798396274724235), np.float64(0.25089896187847854), np.float64(0.08655489332963262), np.float64(0.19483046174517651), np.float64(0.12094499608671105)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs1_classification_subgroup, means_bootstraps_gpvae_obs1_classification_subgroup, standards_deviations_gpvae_obs1_classification_subgroup)

print(lower_bounds_gpvae_obs1_classification_subgroup)
print(upper_bounds_gpvae_obs1_classification_subgroup)

[np.float64(0.19364783590608406), np.float64(0.0070376496942498035), np.float64(0.10522008285767387), np.float64(0.11111241585757914), np.float64(0.1681406336979566)]
[np.float64(0.42094079323709843), np.float64(0.9905435077748617), np.float64(0.44450903008124604), np.float64(0.8748337920812886), np.float64(0.6422363065785395)]


In [ ]:
lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs1_classification_subgroup)

print(lower_bounds_gpvae_obs1_classification_subgroup)
print(upper_bounds_gpvae_obs1_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs1_classification_subgroup, upper_bounds_gpvae_obs1_classification_subgroup)

print(mean_values_ci_gpvae_obs1_classification_subgroup)

[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs2_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_obs2_classification_subgroup, standards_deviations_gpvae_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs2_classification_subgroup)

print(means_bootstraps_gpvae_obs2_classification_subgroup)
print(standards_deviations_gpvae_obs2_classification_subgroup)


[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.3961240813101228), np.float64(0.0), np.float64(0.2858736875756382)]
[np.float64(0.049002577704985846), np.float64(0.09598273849535295), np.float64(1.0268585416236724), np.float64(0.0), np.float64(0.13790114187609237)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs2_classification_subgroup, means_bootstraps_gpvae_obs2_classification_subgroup, standards_deviations_gpvae_obs2_classification_subgroup)

print(lower_bounds_gpvae_obs2_classification_subgroup)
print(upper_bounds_gpvae_obs2_classification_subgroup)

[np.float64(0.1605865061227179), np.float64(-0.062157550347901436), np.float64(-0.6164816774895991), np.float64(0.0), np.float64(0.015592416071548876)]
[np.float64(0.35267308102551326), np.float64(0.3140878708289346), np.float64(3.408729840109845), np.float64(0.0), np.float64(0.5561549590797275)]


In [ ]:
lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs2_classification_subgroup)

print(lower_bounds_gpvae_obs2_classification_subgroup)
print(upper_bounds_gpvae_obs2_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs2_classification_subgroup, upper_bounds_gpvae_obs2_classification_subgroup)

print(mean_values_ci_gpvae_obs2_classification_subgroup)

[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.396124081310123), np.float64(0.0), np.float64(0.28587368757563814)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [1,31,6,0,32]

results_bootstrap_gpvae_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_gpvae_obs3_classification_subgroup.append(toolkits.bootstrap(testing_ae_gpvae_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_gpvae_obs3_classification_subgroup)

print(means_bootstraps_gpvae_obs3_classification_subgroup)
print(standards_deviations_gpvae_obs3_classification_subgroup)


[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.20886002502599546)]
[np.float64(0.050281311239894225), np.float64(0.0), np.float64(0.8237478596037178), np.float64(0.19004214803335054), np.float64(0.12978497928121874)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_gpvae_obs3_classification_subgroup, means_bootstraps_gpvae_obs3_classification_subgroup, standards_deviations_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

[np.float64(0.05300461518183637), np.float64(0.0), np.float64(-0.4551198683898825), np.float64(-0.12823364717905072), np.float64(-0.045513860099470366)]
[np.float64(0.25010373343311976), np.float64(0.0), np.float64(2.773912405940605), np.float64(0.6167178842009423), np.float64(0.4632339101514613)]


In [ ]:
lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_gpvae_obs3_classification_subgroup)

print(lower_bounds_gpvae_obs3_classification_subgroup)
print(upper_bounds_gpvae_obs3_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_gpvae_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_gpvae_obs3_classification_subgroup, upper_bounds_gpvae_obs3_classification_subgroup)

print(mean_values_ci_gpvae_obs3_classification_subgroup)

[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.2088600250259955)]


## MRNN

### Calculate MAE and AE

In [ ]:
testing_mae_mrnn_variables_standard, testing_ae_mrnn_variables_standard = toolkits.calculate_mae(mrnn_imputation_standard, test_X_ori_variable_standard_norm, indicating_mask_variable_standard_norm)

### Taking the MAE difference for MRNN model

#### Difference between gender group

In [ ]:
diff_mrnn_gender_subgroup = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 1, 2, variables)
diff_mrnn_gender_subgroup

[('Cholesterol', 0.46007480562614733),
 ('SaO2', 0.23089925994991106),
 ('Weight', 0.1781846451592244),
 ('AST', 0.1741083024220948),
 ('PaCO2', 0.12962865455656114)]

#### Difference between age group

In [ ]:
diff_mrnn_age_subgroup = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 4, 5, variables)
diff_mrnn_age_subgroup

[('Cholesterol', 0.5264706140850112),
 ('ALP', 0.42118039343106295),
 ('SaO2', 0.21218979884570444),
 ('TroponinI', 0.18821601789959796),
 ('Bilirubin', 0.1405204793192777)]

#### Difference between ICUType group

In [ ]:
diff_mrnn_ICUType_1_2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 7, variables)
diff_mrnn_ICUType_1_3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 8, variables)
diff_mrnn_ICUType_1_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 6, 9, variables)
diff_mrnn_ICUType_2_3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 7, 8, variables)
diff_mrnn_ICUType_2_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 7, 9, variables)
diff_mrnn_ICUType_3_4 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 8, 9, variables)

In [ ]:
print("Diferença(MAE) ICUType 1 e 2")
print("----------------------------")
print(diff_mrnn_ICUType_1_2)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 3")
print("----------------------------")
print(diff_mrnn_ICUType_1_3)
print("----------------------------")
print("Diferença(MAE) ICUType 1 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_1_4)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 3")
print(diff_mrnn_ICUType_2_3)
print("----------------------------")
print("Diferença(MAE) ICUType 2 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_2_4)
print("----------------------------")
print("Diferença(MAE) ICUType 3 e 4")
print("----------------------------")
print(diff_mrnn_ICUType_3_4)
print("----------------------------")

Diferença(MAE) ICUType 1 e 2
----------------------------
[('Cholesterol', 0.9010319943785403), ('TroponinI', 0.8885610723184879), ('TroponinT', 0.2790266043092085), ('AST', 0.2552394455397726), ('HR', 0.24475318001332502)]
----------------------------
Diferença(MAE) ICUType 1 e 3
----------------------------
[('SaO2', 1.126906434849816), ('ALP', 0.826833726615776), ('TroponinI', 0.6275078131032561), ('PaCO2', 0.3166885871390994), ('K', 0.24651464330000916)]
----------------------------
Diferença(MAE) ICUType 1 e 4
----------------------------
[('TroponinI', 0.4752300632564468), ('TroponinT', 0.28805061664771353), ('Albumin', 0.2061163193636698), ('Glucose', 0.20571705409253582), ('Cholesterol', 0.20496776200009514)]
----------------------------
Diferença(MAE) ICUType 2 e 3
[('SaO2', 1.3273285721076542), ('Cholesterol', 0.7254168014070564), ('ALP', 0.6559594688208507), ('HCO3', 0.444614029946391), ('PaCO2', 0.4402514603625217)]
----------------------------
Diferença(MAE) ICUType 2 e 4


In [ ]:
print("Top 5 variáveis ICUType")
print("-----------------------")
print(variables[28])
print(variables[6])
print(variables[31])
print(variables[0])
print(variables[24])

Top 5 variáveis ICUType
-----------------------
SaO2
Cholesterol
TroponinI
ALP
PaCO2


#### Difference between BMI group

In [ ]:
diff_mrnn_classification_undefined_low = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 11, variables)
diff_mrnn_classification_undefined_normal = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 12, variables)
diff_mrnn_classification_undefined_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 13, variables)
diff_mrnn_classification_undefined_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 14, variables)
diff_mrnn_classification_undefined_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 15, variables)
diff_mrnn_classification_undefined_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 10, 16, variables)

diff_mrnn_classification_low_normal = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 12, variables)
diff_mrnn_classification_low_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 13, variables)
diff_mrnn_classification_low_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 14, variables)
diff_mrnn_classification_low_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 15, variables)
diff_mrnn_classification_low_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 11, 16, variables)

diff_mrnn_classification_normal_over = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 13, variables)
diff_mrnn_classification_normal_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 14, variables)
diff_mrnn_classification_normal_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 15, variables)
diff_mrnn_classification_normal_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 12, 16, variables)

diff_mrnn_classification_over_obs1 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 14, variables)
diff_mrnn_classification_over_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 15, variables)
diff_mrnn_classification_over_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 13, 16, variables)


diff_mrnn_classification_obs1_obs2 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 14, 15, variables)
diff_mrnn_classification_obs1_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 14, 16, variables)

diff_mrnn_classification_obs2_obs3 = toolkits.diff_mae_top_5(testing_mae_mrnn_variables_standard, 15, 16, variables)

In [ ]:
print("Diferença(MAE) Classification Undefined and low")
print("----------------------------")
print(diff_mrnn_classification_undefined_low)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and normal")
print("----------------------------")
print(diff_mrnn_classification_undefined_normal)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and over")
print("----------------------------")
print(diff_mrnn_classification_undefined_over)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs1")
print(diff_mrnn_classification_undefined_obs1)
print("----------------------------")
print("Diferença(MAE) Classification Undefined and obs2")
print("----------------------------")
print(diff_mrnn_classification_undefined_obs2)
print("----------------------------")
print("Diferença(MAE) ICUType Undefined and obs3")
print("----------------------------")
print(diff_mrnn_classification_undefined_obs3)
print("----------------------------")

Diferença(MAE) Classification Undefined and low
----------------------------
[('ALP', 1.7276133990683362), ('Bilirubin', 0.9914312934423044), ('TroponinI', 0.7605306891067674), ('Weight', 0.5541701194492721), ('Cholesterol', 0.4686585973942653)]
----------------------------
Diferença(MAE) Classification Undefined and normal
----------------------------
[('Cholesterol', 0.3793471636845912), ('PaCO2', 0.2607229442068275), ('SaO2', 0.23204288622686337), ('TroponinI', 0.19152385318395748), ('HCO3', 0.18600797369321342)]
----------------------------
Diferença(MAE) Classification Undefined and over
----------------------------
[('Cholesterol', 0.7311865583297629), ('TroponinI', 0.4223997471662162), ('Weight', 0.3506808760264265), ('PaCO2', 0.2270916455603793), ('SaO2', 0.2137441109303908)]
----------------------------
Diferença(MAE) Classification Undefined and obs1
[('Ph', 0.8858111069773932), ('PaCO2', 0.2961064224948988), ('HCO3', 0.25893221785255016), ('SaO2', 0.2587867211362769), ('Albu

In [ ]:
print("Diferença(MAE) Classification low and normal")
print("----------------------------")
print(diff_mrnn_classification_low_normal)
print("----------------------------")
print("Diferença(MAE) Classification low and over")
print("----------------------------")
print(diff_mrnn_classification_low_over)
print("----------------------------")
print("Diferença(MAE) Classification low and obs1")
print("----------------------------")
print(diff_mrnn_classification_low_obs1)
print("----------------------------")
print("Diferença(MAE) Classification low and obs2")
print(diff_mrnn_classification_low_obs2)
print("----------------------------")
print("Diferença(MAE) Classification low and obs3")
print("----------------------------")
print(diff_mrnn_classification_low_obs3)
print("----------------------------")

Diferença(MAE) Classification low and normal
----------------------------
[('ALP', 1.616655613152504), ('Bilirubin', 0.9207620318461242), ('Cholesterol', 0.8480057610788565), ('TroponinI', 0.5690068359228099), ('HCO3', 0.5367857762848458)]
----------------------------
Diferença(MAE) Classification low and over
----------------------------
[('ALP', 1.8004731135079597), ('Cholesterol', 1.1998451557240282), ('TroponinI', 1.1829304362729836), ('Bilirubin', 1.1149758355325403), ('Weight', 0.9048509954756986)]
----------------------------
Diferença(MAE) Classification low and obs1
----------------------------
[('ALP', 1.839342476713079), ('Bilirubin', 1.1198139228432797), ('Ph', 0.8807265511276338), ('Weight', 0.782381102471485), ('TroponinI', 0.6293802804910801)]
----------------------------
Diferença(MAE) Classification low and obs2
[('ALP', 1.701601911486742), ('Bilirubin', 1.0552999585115248), ('TroponinT', 0.9759888843088509), ('HCO3', 0.6339041283774007), ('Weight', 0.5288315136741774)

In [ ]:
print("Diferença(MAE) Classification normal and over")
print("----------------------------")
print(diff_mrnn_classification_normal_over)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs1")
print("----------------------------")
print(diff_mrnn_classification_normal_obs1)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs2")
print("----------------------------")
print(diff_mrnn_classification_normal_obs2)
print("----------------------------")
print("Diferença(MAE) Classification normal and obs3")
print(diff_mrnn_classification_normal_obs3)
print("----------------------------")

Diferença(MAE) Classification normal and over
----------------------------
[('TroponinI', 0.6139236003501737), ('Weight', 0.39083485356388614), ('Cholesterol', 0.35183939464517167), ('ALT', 0.25331368516240216), ('Bilirubin', 0.19421380368641616)]
----------------------------
Diferença(MAE) Classification normal and obs1
----------------------------
[('Ph', 0.8908243780585775), ('Cholesterol', 0.3310031978642243), ('Weight', 0.2683649605596725), ('ALP', 0.22268686356057493), ('Albumin', 0.1992342096852764)]
----------------------------
Diferença(MAE) Classification normal and obs2
----------------------------
[('TroponinT', 0.9827645580136316), ('Cholesterol', 0.8717828192132384), ('TroponinI', 0.5690068359228099), ('ALT', 0.2871264264440603), ('Creatinine', 0.2062260662166976)]
----------------------------
Diferença(MAE) Classification normal and obs3
[('ALT', 1.0982371420715205), ('Cholesterol', 0.953380515668881), ('Weight', 0.86452363699577), ('TroponinT', 0.5796770889726964), ('HC

In [ ]:
print("Diferença(MAE) Classification over and obs1")
print("----------------------------")
print(diff_mrnn_classification_over_obs1)
print("----------------------------")
print("Diferença(MAE) Classification over and obs2")
print("----------------------------")
print(diff_mrnn_classification_over_obs2)
print("----------------------------")
print("Diferença(MAE) Classification over and obs3")
print("----------------------------")
print(diff_mrnn_classification_over_obs3)
print("----------------------------")

Diferença(MAE) Classification over and obs1
----------------------------
[('Ph', 0.8888769174492269), ('Cholesterol', 0.6828425925093959), ('TroponinI', 0.5535501557819035), ('ALT', 0.2339750215206805), ('TroponinT', 0.23237983193846368)]
----------------------------
Diferença(MAE) Classification over and obs2
----------------------------
[('Cholesterol', 1.22362221385841), ('TroponinI', 1.1829304362729836), ('TroponinT', 0.8424714467402539), ('Weight', 0.3760194818015212), ('RespRate', 0.20419277047251283)]
----------------------------
Diferença(MAE) Classification over and obs3
----------------------------
[('Cholesterol', 1.3052199103140527), ('Weight', 1.2553584905596562), ('ALT', 0.8449234569091182), ('TroponinI', 0.8326977045645636), ('TroponinT', 0.4393839776993187)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs1 and obs2 ")
print("----------------------------")
print(diff_mrnn_classification_obs1_obs2)
print("----------------------------")
print("Diferença(MAE) Classification obs1 and obs3 ")
print("----------------------------")
print(diff_mrnn_classification_obs1_obs3)
print("----------------------------")

Diferença(MAE) Classification obs1 and obs2 
----------------------------
[('TroponinT', 1.0748512786787177), ('Ph', 0.8644289951196773), ('TroponinI', 0.6293802804910801), ('Cholesterol', 0.5407796213490141), ('ALT', 0.2677877628023386)]
----------------------------
Diferença(MAE) Classification obs1 and obs3 
----------------------------
[('Weight', 1.1328885975554424), ('ALT', 1.0788984784297986), ('Ph', 0.884502858503702), ('TroponinT', 0.6717638096377824), ('Cholesterol', 0.6223773178046568)]
----------------------------


In [ ]:
print("Diferença(MAE) Classification obs2 and obs3 ")
print("----------------------------")
print(diff_mrnn_classification_obs2_obs3)
print("----------------------------")

Diferença(MAE) Classification obs2 and obs3 
----------------------------
[('Weight', 0.8793390087581349), ('ALT', 0.8111107156274601), ('TroponinT', 0.4030874690409352), ('HCO3', 0.40178041918761953), ('TroponinI', 0.35023273170841995)]
----------------------------


In [ ]:
print("Top 5 variáveis BMI")
print("-----------------------")
print(variables[6])
print(variables[0])
print(variables[31])
print(variables[35])
print(variables[32])

Top 5 variáveis BMI
-----------------------
Cholesterol
ALP
TroponinI
Weight
TroponinT


### Confidence Interval (Female Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,28,35,2,24]

results_bootstrap_mrnn_female = []

for i in indices:
       results_bootstrap_mrnn_female.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 1, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_female,standards_deviations_mrnn_female = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_female)

print(means_bootstraps_mrnn_female)
print(standards_deviations_mrnn_female)


[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]
[np.float64(0.3381506628642691), np.float64(0.08322467917877542), np.float64(0.07193090333728737), np.float64(0.02436604729618584), np.float64(0.03851788865527756)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_female, upper_bounds_mrnn_female = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_female, means_bootstraps_mrnn_female,standards_deviations_mrnn_female)

print(lower_bounds_mrnn_female)
print(upper_bounds_mrnn_female)

[np.float64(0.5235200070760525), np.float64(0.1366787734720611), np.float64(0.3825933708508273), np.float64(0.20450627248860936), np.float64(0.3499327009241326)]
[np.float64(1.849046248200679), np.float64(0.46291352110266176), np.float64(0.6645573306838577), np.float64(0.30001942278085697), np.float64(0.5009200499738685)]


In [ ]:
lower_bounds_mrnn_female, upper_bounds_mrnn_female = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_female)

print(lower_bounds_mrnn_female)
print(upper_bounds_mrnn_female)

#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_mrnn_female = toolkits.calc_mean_values_ci(lower_bounds_mrnn_female, upper_bounds_mrnn_female)

print(mean_values_ci_mrnn_female)

[np.float64(1.1862831276383659), np.float64(0.2997961472873614), np.float64(0.5235753507673425), np.float64(0.25226284763473317), np.float64(0.4254263754490005)]


### Confidence Interval (Male Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,28,35,2,24]

results_bootstrap_mrnn_male = []

for i in indices:
       results_bootstrap_mrnn_male.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 2, i, 9000))

#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_male,standards_deviations_mrnn_male = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_male)

print(means_bootstraps_mrnn_male)
print(standards_deviations_mrnn_male)


[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]
[np.float64(0.17358454405745072), np.float64(0.025632322705534415), np.float64(0.2124414858781128), np.float64(0.03724148791663912), np.float64(0.04259881898681301)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_male, upper_bounds_mrnn_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_male, means_bootstraps_mrnn_male,standards_deviations_mrnn_male)

print(lower_bounds_mrnn_male)
print(upper_bounds_mrnn_male)

[np.float64(0.5129891453019042), np.float64(0.07149955530644256), np.float64(0.2837910673267833), np.float64(0.33405772637289466), np.float64(0.4386664730964766)]
[np.float64(1.1934280545527236), np.float64(0.17197641399235403), np.float64(1.1165463896133345), np.float64(0.4800416764674872), np.float64(0.6056507750926656)]


In [ ]:
lower_bounds_mrnn_male, upper_bounds_mrnn_male = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_male)

print(lower_bounds_mrnn_male)
print(upper_bounds_mrnn_male)

#### Mean values of lower bound and upper bound


In [ ]:
mean_values_ci_mrnn_male = toolkits.calc_mean_values_ci(lower_bounds_mrnn_male, upper_bounds_mrnn_male)

print(mean_values_ci_mrnn_male)

[np.float64(0.8532085999273139), np.float64(0.12173798464939829), np.float64(0.7001687284700588), np.float64(0.4070497014201909), np.float64(0.5221586240945711)]


### Confidence Interval (+65 Subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,28,31,5]

results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup = []

for i in indices:
       results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 4, i, 9000))


#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup,standards_deviations_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup)

print(means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup)
print(standards_deviations_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.9260057824625499), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]
[np.float64(0.14022222850550667), np.float64(0.2902139592129938), np.float64(0.025161049870449845), np.float64(0.08795901770065773), np.float64(0.06166376517595487)]


#### Calculating lower bound and upper bound


In [ ]:
lower_bounds_mrnn_more_than_or_equal_to_65_subgroup, upper_bounds_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup, means_bootstraps_mrnn_more_than_or_equal_to_65_subgroup,standards_deviations_mrnn_more_than_or_equal_to_65_subgroup)

print(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup)
print(upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)


[np.float64(0.3602797122452329), np.float64(0.3571968745943058), np.float64(0.18962243507704596), np.float64(0.491014463045211), np.float64(0.45334176332908904)]
[np.float64(0.9099407476507105), np.float64(1.4948146903307942), np.float64(0.2882519381956417), np.float64(0.8358074766628315), np.float64(0.6950592811211025)]


In [ ]:
lower_bounds_mrnn_more_than_or_equal_to_65_subgroup, upper_bounds_mrnn_more_than_or_equal_to_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_more_than_or_equal_to_65_subgroup)

print(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup)
print(upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_more_than_or_equal_to_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_more_than_or_equal_to_65_subgroup , upper_bounds_mrnn_more_than_or_equal_to_65_subgroup)

print(mean_values_ci_mrnn_more_than_or_equal_to_65_subgroup)

[np.float64(0.6351102299479717), np.float64(0.92600578246255), np.float64(0.23893718663634383), np.float64(0.6634109698540213), np.float64(0.5742005222250958)]


### Confidence Interval (-65 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,28,31,5]

results_bootstrap_mrnn_less_than_65_subgroup = []

for i in indices:
       results_bootstrap_mrnn_less_than_65_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 5, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_less_than_65_subgroup,standards_deviations_mrnn_less_than_65_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_less_than_65_subgroup)

print(means_bootstraps_mrnn_less_than_65_subgroup)
print(standards_deviations_mrnn_less_than_65_subgroup)


[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]
[np.float64(0.4337083267621267), np.float64(0.06965814575847884), np.float64(0.08711583404267569), np.float64(0.6101374424469065), np.float64(0.06334923482333744)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_less_than_65_subgroup, means_bootstraps_mrnn_less_than_65_subgroup, standards_deviations_mrnn_less_than_65_subgroup)

print(lower_bounds_mrnn_less_than_65_subgroup)
print(upper_bounds_mrnn_less_than_65_subgroup)

[np.float64(0.23705163755647063), np.float64(0.40032284839995347), np.float64(0.30726428110681003), np.float64(-0.41507045428487355), np.float64(0.5887783496794352)]
[np.float64(1.9371570380542658), np.float64(0.6733777622328736), np.float64(0.6487520755204355), np.float64(1.9766243713457599), np.float64(0.8371027870832592)]


In [ ]:
lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_less_than_65_subgroup)

print(lower_bounds_mrnn_less_than_65_subgroup)
print(upper_bounds_mrnn_less_than_65_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_less_than_65_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_less_than_65_subgroup, upper_bounds_mrnn_less_than_65_subgroup)

print(mean_values_ci_mrnn_less_than_65_subgroup)

[np.float64(1.0871043378053682), np.float64(0.5368503053164135), np.float64(0.47800817831362274), np.float64(0.7807769585304432), np.float64(0.7129405683813472)]


### Confidence Interval (ICUType 1 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,6,31,0,24]

results_bootstrap_mrnn_ICUType_1_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_1_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 6, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_1_subgroup, standards_deviations_mrnn_ICUType_1_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_1_subgroup)

print(means_bootstraps_mrnn_ICUType_1_subgroup)
print(standards_deviations_mrnn_ICUType_1_subgroup)


[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547066), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]
[np.float64(0.17812570383563214), np.float64(0.03508402734675316), np.float64(0.2710993234914266), np.float64(0.37708432885457466), np.float64(0.15436752793597142)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_1_subgroup, means_bootstraps_mrnn_ICUType_1_subgroup, standards_deviations_mrnn_ICUType_1_subgroup)

print(lower_bounds_mrnn_ICUType_1_subgroup)
print(upper_bounds_mrnn_ICUType_1_subgroup)

[np.float64(0.1735625173126043), np.float64(0.1565126112697423), np.float64(0.437748337678337), np.float64(0.6022929596856587), np.float64(0.41865583030294223)]
[np.float64(0.8718024457899787), np.float64(0.29403947133425135), np.float64(1.5004381582310762), np.float64(2.080436367064507), np.float64(1.0237654205769116)]


In [ ]:
lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_1_subgroup)

print(lower_bounds_mrnn_ICUType_1_subgroup)
print(upper_bounds_mrnn_ICUType_1_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_1_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_1_subgroup, upper_bounds_mrnn_ICUType_1_subgroup)

print(mean_values_ci_mrnn_ICUType_1_subgroup)

[np.float64(0.5226824815512915), np.float64(0.22527604130199683), np.float64(0.9690932479547065), np.float64(1.3413646633750829), np.float64(0.7212106254399269)]


### Confidence Interval (ICUType 2 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,6,31,0,24]

results_bootstrap_mrnn_ICUType_2_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_2_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 7, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_2_subgroup, standards_deviations_mrnn_ICUType_2_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_2_subgroup)

print(means_bootstraps_mrnn_ICUType_2_subgroup)
print(standards_deviations_mrnn_ICUType_2_subgroup) 


[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.3769132654783525), np.float64(0.17130476566918515)]
[np.float64(0.24746020204511532), np.float64(0.02844864967364565), np.float64(0.0), np.float64(0.12241202033633213), np.float64(0.04748266503243493)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_2_subgroup, means_bootstraps_mrnn_ICUType_2_subgroup, standards_deviations_mrnn_ICUType_2_subgroup)

print(lower_bounds_mrnn_ICUType_2_subgroup)
print(upper_bounds_mrnn_ICUType_2_subgroup)

NameError: name 'toolkits' is not defined

In [ ]:
lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_2_subgroup)

print(lower_bounds_mrnn_ICUType_2_subgroup)
print(upper_bounds_mrnn_ICUType_2_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_2_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_2_subgroup, upper_bounds_mrnn_ICUType_2_subgroup)

print(mean_values_ci_mrnn_ICUType_2_subgroup)

[np.float64(0.5628893404476448), np.float64(0.253544682516006), np.float64(0.0), np.float64(0.37691326547835247), np.float64(0.17130476566918512)]


### Confidence Interval (ICUType 3 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,6,31,0,24]

results_bootstrap_mrnn_ICUType_3_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_3_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 8, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_3_subgroup, standards_deviations_mrnn_ICUType_3_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_3_subgroup)

print(means_bootstraps_mrnn_ICUType_3_subgroup)
print(standards_deviations_mrnn_ICUType_3_subgroup)


[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]
[np.float64(0.4559261164801034), np.float64(0.45097433675337983), np.float64(0.1878586172758027), np.float64(0.06133680568713781), np.float64(0.03548543506753726)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_3_subgroup, means_bootstraps_mrnn_ICUType_3_subgroup, standards_deviations_mrnn_ICUType_3_subgroup)

print(lower_bounds_mrnn_ICUType_3_subgroup)
print(upper_bounds_mrnn_ICUType_3_subgroup)

[np.float64(0.6996377293253487), np.float64(0.25263761572853904), np.float64(0.3276231392998342), np.float64(0.4209399971546227), np.float64(0.16790862349933133)]
[np.float64(2.4868352651497814), np.float64(2.020424531705464), np.float64(1.0640153873919687), np.float64(0.661375857301666), np.float64(0.30700897291554674)]


In [ ]:
lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_3_subgroup)

print(lower_bounds_mrnn_ICUType_3_subgroup)
print(upper_bounds_mrnn_ICUType_3_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_3_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_3_subgroup, upper_bounds_mrnn_ICUType_3_subgroup)

print(mean_values_ci_mrnn_ICUType_3_subgroup)

[np.float64(1.593236497237565), np.float64(1.1365310737170016), np.float64(0.6958192633459015), np.float64(0.5411579272281444), np.float64(0.23745879820743904)]


### Confidence Interval (ICUType 4 subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [28,6,31,0,24]

results_bootstrap_mrnn_ICUType_4_subgroup = []

for i in indices:
       results_bootstrap_mrnn_ICUType_4_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 9, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_ICUType_4_subgroup, standards_deviations_mrnn_ICUType_4_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_ICUType_4_subgroup)

print(means_bootstraps_mrnn_ICUType_4_subgroup)
print(standards_deviations_mrnn_ICUType_4_subgroup)


[np.float64(0.44306473111336137), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]
[np.float64(0.0935566892280583), np.float64(0.08074364451634707), np.float64(0.2681495472441715), np.float64(0.2724504216824265), np.float64(0.022189091660584512)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_ICUType_4_subgroup, means_bootstraps_mrnn_ICUType_4_subgroup, standards_deviations_mrnn_ICUType_4_subgroup)

print(lower_bounds_mrnn_ICUType_4_subgroup)
print(upper_bounds_mrnn_ICUType_4_subgroup)

[np.float64(0.25969698971356064), np.float64(0.33294655262192163), np.float64(0.2031662976673827), np.float64(0.1584331675516304), np.float64(0.13677304273715427)]
[np.float64(0.626432472513162), np.float64(0.6494558230870122), np.float64(1.2542932078059783), np.float64(1.2264191956922437), np.float64(0.22375268374596166)]


In [ ]:
lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_ICUType_4_subgroup)

print(lower_bounds_mrnn_ICUType_4_subgroup)
print(upper_bounds_mrnn_ICUType_4_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_ICUType_4_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_ICUType_4_subgroup, upper_bounds_mrnn_ICUType_4_subgroup)

print(mean_values_ci_mrnn_ICUType_4_subgroup)

[np.float64(0.4430647311133613), np.float64(0.4912011878544669), np.float64(0.7287297527366805), np.float64(0.692426181621937), np.float64(0.18026286324155796)]


### Confidence Interval (Undefined classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_undefined_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_undefined_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 10, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_undefined_classification_subgroup, standards_deviations_mrnn_undefined_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_undefined_classification_subgroup)

print(means_bootstraps_mrnn_undefined_classification_subgroup)
print(standards_deviations_mrnn_undefined_classification_subgroup)


[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]
[np.float64(0.05986934359644469), np.float64(0.17840974136243826), np.float64(0.044640773548291844), np.float64(0.2853216956527157), np.float64(0.12549413739177268)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_undefined_classification_subgroup, means_bootstraps_mrnn_undefined_classification_subgroup, standards_deviations_mrnn_undefined_classification_subgroup)

print(lower_bounds_mrnn_undefined_classification_subgroup)
print(upper_bounds_mrnn_undefined_classification_subgroup)

[np.float64(0.28951948015518403), np.float64(0.22394384394790923), np.float64(0.18842469528371564), np.float64(0.28379016616066055), np.float64(0.16723269590166293)]
[np.float64(0.5242029946093546), np.float64(0.9232971790708792), np.float64(0.3634133120770363), np.float64(1.4022306611351032), np.float64(0.6591606750192545)]


In [ ]:
lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_undefined_classification_subgroup)

print(lower_bounds_mrnn_undefined_classification_subgroup)
print(upper_bounds_mrnn_undefined_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_undefined_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_undefined_classification_subgroup, upper_bounds_mrnn_undefined_classification_subgroup)

print(mean_values_ci_mrnn_undefined_classification_subgroup)

[np.float64(0.4068612373822693), np.float64(0.5736205115093942), np.float64(0.275919003680376), np.float64(0.8430104136478819), np.float64(0.4131966854604587)]


### Confidence Interval (Low weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_low_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_low_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 11, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_low_weight_classification_subgroup, standards_deviations_mrnn_low_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_low_weight_classification_subgroup)

print(means_bootstraps_mrnn_low_weight_classification_subgroup)
print(standards_deviations_mrnn_low_weight_classification_subgroup)


[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032636), np.float64(0.0), np.float64(1.4606367473586466)]
[np.float64(1.666676645356208), np.float64(0.05309602591127397), np.float64(0.3139178166078663), np.float64(0.0), np.float64(1.2916342659860158)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_low_weight_classification_subgroup, means_bootstraps_mrnn_low_weight_classification_subgroup, standards_deviations_mrnn_low_weight_classification_subgroup)

print(lower_bounds_mrnn_low_weight_classification_subgroup)
print(upper_bounds_mrnn_low_weight_classification_subgroup)

[np.float64(-1.2543455052071963), np.float64(-0.03473297672088424), np.float64(-0.2048614788536039), np.float64(0.0), np.float64(-1.070919895171773)]
[np.float64(5.278906892337211), np.float64(0.17339962029572073), np.float64(1.0256737504601312), np.float64(0.0), np.float64(3.9921933898890662)]


In [ ]:
lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_low_weight_classification_subgroup)

print(lower_bounds_mrnn_low_weight_classification_subgroup)
print(upper_bounds_mrnn_low_weight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_low_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_low_weight_classification_subgroup, upper_bounds_mrnn_low_weight_classification_subgroup)

print(mean_values_ci_mrnn_low_weight_classification_subgroup)

[np.float64(2.0122806935650077), np.float64(0.06933332178741825), np.float64(0.4104061358032637), np.float64(0.0), np.float64(1.4606367473586466)]


### Confidence Interval (Normal weight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_normal_weight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_normal_weight_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 12, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_normal_weight_classification_subgroup, standards_deviations_mrnn_normal_weight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_normal_weight_classification_subgroup)

print(means_bootstraps_mrnn_normal_weight_classification_subgroup)
print(standards_deviations_mrnn_normal_weight_classification_subgroup)


[np.float64(0.548686639105505), np.float64(0.7625897872503594), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]
[np.float64(0.12623025707018218), np.float64(0.3731308327837512), np.float64(0.07341434562099826), np.float64(0.089947135630567), np.float64(0.2013724566774524)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_normal_weight_classification_subgroup, means_bootstraps_mrnn_normal_weight_classification_subgroup, standards_deviations_mrnn_normal_weight_classification_subgroup)

print(lower_bounds_mrnn_normal_weight_classification_subgroup)
print(upper_bounds_mrnn_normal_weight_classification_subgroup)

[np.float64(0.3012798814887154), np.float64(0.03126679347276984), np.float64(0.10543028800815332), np.float64(0.38859566088565467), np.float64(0.12860300267586205)]
[np.float64(0.7960933967222945), np.float64(1.4939127810279489), np.float64(0.3932092347396181), np.float64(0.7411819535825561), np.float64(0.9179685278081802)]


In [ ]:
lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_normal_weight_classification_subgroup)

print(lower_bounds_mrnn_normal_weight_classification_subgroup)
print(upper_bounds_mrnn_normal_weight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_normal_weight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_normal_weight_classification_subgroup, upper_bounds_mrnn_normal_weight_classification_subgroup)

print(mean_values_ci_mrnn_normal_weight_classification_subgroup)

[np.float64(0.548686639105505), np.float64(0.7625897872503593), np.float64(0.24931976137388573), np.float64(0.5648888072341054), np.float64(0.5232857652420211)]


### Confidence Interval (Overweight classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_overweight_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_overweight_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 13, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_overweight_classification_subgroup, standards_deviations_mrnn_overweight_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_overweight_classification_subgroup)

print(means_bootstraps_mrnn_overweight_classification_subgroup)
print(standards_deviations_mrnn_overweight_classification_subgroup)


[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082165), np.float64(0.1969866594273391)]
[np.float64(0.05690905629749502), np.float64(0.2748051570149171), np.float64(0.190452089816047), np.float64(0.5659200713279424), np.float64(0.06465085722820384)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_overweight_classification_subgroup, means_bootstraps_mrnn_overweight_classification_subgroup, standards_deviations_mrnn_overweight_classification_subgroup)

print(lower_bounds_mrnn_overweight_classification_subgroup)
print(upper_bounds_mrnn_overweight_classification_subgroup)

[np.float64(0.16821060322753356), np.float64(0.9483102156492423), np.float64(0.07116983979697916), np.float64(-0.06774702542288913), np.float64(0.07027330769041853)]
[np.float64(0.3912900047020388), np.float64(2.0255266366794666), np.float64(0.8177283134366586), np.float64(2.150618890439322), np.float64(0.3237000111642596)]


In [ ]:
lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_overweight_classification_subgroup)

print(lower_bounds_mrnn_overweight_classification_subgroup)
print(upper_bounds_mrnn_overweight_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_overweight_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_overweight_classification_subgroup, upper_bounds_mrnn_overweight_classification_subgroup)

print(mean_values_ci_mrnn_overweight_classification_subgroup)

[np.float64(0.2797503039647862), np.float64(1.4869184261643544), np.float64(0.44444907661681887), np.float64(1.0414359325082163), np.float64(0.19698665942733906)]


### Confidence Interval (Obesity 1 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_obs1_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs1_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 14, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_obs1_classification_subgroup, standards_deviations_mrnn_obs1_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs1_classification_subgroup)

print(means_bootstraps_mrnn_obs1_classification_subgroup)
print(standards_deviations_mrnn_obs1_classification_subgroup)


[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]
[np.float64(0.05798396274724235), np.float64(0.25089896187847854), np.float64(0.08655489332963262), np.float64(0.19483046174517651), np.float64(0.12094499608671105)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs1_classification_subgroup, means_bootstraps_mrnn_obs1_classification_subgroup, standards_deviations_mrnn_obs1_classification_subgroup)

print(lower_bounds_mrnn_obs1_classification_subgroup)
print(upper_bounds_mrnn_obs1_classification_subgroup)

[np.float64(0.19364783590608406), np.float64(0.0070376496942498035), np.float64(0.10522008285767387), np.float64(0.11111241585757914), np.float64(0.1681406336979566)]
[np.float64(0.42094079323709843), np.float64(0.9905435077748617), np.float64(0.44450903008124604), np.float64(0.8748337920812886), np.float64(0.6422363065785395)]


In [ ]:
lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs1_classification_subgroup)

print(lower_bounds_mrnn_obs1_classification_subgroup)
print(upper_bounds_mrnn_obs1_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_obs1_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs1_classification_subgroup, upper_bounds_mrnn_obs1_classification_subgroup)

print(mean_values_ci_mrnn_obs1_classification_subgroup)

[np.float64(0.30729431457159123), np.float64(0.49879057873455573), np.float64(0.27486455646945995), np.float64(0.4929731039694339), np.float64(0.40518847013824805)]


### Confidence Interval (Obesity 2 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_obs2_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs2_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 15, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_obs2_classification_subgroup, standards_deviations_mrnn_obs2_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs2_classification_subgroup)

print(means_bootstraps_mrnn_obs2_classification_subgroup)
print(standards_deviations_mrnn_obs2_classification_subgroup) 


[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.3961240813101228), np.float64(0.0), np.float64(0.2858736875756382)]
[np.float64(0.049002577704985846), np.float64(0.09598273849535295), np.float64(1.0268585416236724), np.float64(0.0), np.float64(0.13790114187609237)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs2_classification_subgroup, means_bootstraps_mrnn_obs2_classification_subgroup, standards_deviations_mrnn_obs2_classification_subgroup)

print(lower_bounds_mrnn_obs2_classification_subgroup)
print(upper_bounds_mrnn_obs2_classification_subgroup)

[np.float64(0.1605865061227179), np.float64(-0.062157550347901436), np.float64(-0.6164816774895991), np.float64(0.0), np.float64(0.015592416071548876)]
[np.float64(0.35267308102551326), np.float64(0.3140878708289346), np.float64(3.408729840109845), np.float64(0.0), np.float64(0.5561549590797275)]


In [ ]:
lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs2_classification_subgroup)

print(lower_bounds_mrnn_obs2_classification_subgroup)
print(upper_bounds_mrnn_obs2_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_obs2_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs2_classification_subgroup, upper_bounds_mrnn_obs2_classification_subgroup)

print(mean_values_ci_mrnn_obs2_classification_subgroup)

[np.float64(0.2566297935741156), np.float64(0.12596516024051657), np.float64(1.396124081310123), np.float64(0.0), np.float64(0.28587368757563814)]


### Confidence Interval (Obesity 3 classification subgroup)

#### Distribution Bootstrap for the Top 5 variables with Largest Differences in MAE (Mean Absolute Error) 

In [ ]:
indices = [6,0,31,35,32]

results_bootstrap_mrnn_obs3_classification_subgroup = []

for i in indices:
       results_bootstrap_mrnn_obs3_classification_subgroup.append(toolkits.bootstrap(testing_ae_mrnn_variables_standard, indicating_mask_variable_standard_norm, 16, i, 9000))



#### Calculating the mean and standard deviation of the bootstrap distribution

In [ ]:
means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup = toolkits.calc_mean_and_standard_deviation(results_bootstrap_mrnn_obs3_classification_subgroup)

print(means_bootstraps_mrnn_obs3_classification_subgroup)
print(standards_deviations_mrnn_obs3_classification_subgroup)


[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.20886002502599546)]
[np.float64(0.050281311239894225), np.float64(0.0), np.float64(0.8237478596037178), np.float64(0.19004214803335054), np.float64(0.12978497928121874)]


#### Calculating lower bound and upper bound

In [ ]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound(results_bootstrap_mrnn_obs3_classification_subgroup, means_bootstraps_mrnn_obs3_classification_subgroup, standards_deviations_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

[np.float64(0.05300461518183637), np.float64(0.0), np.float64(-0.4551198683898825), np.float64(-0.12823364717905072), np.float64(-0.045513860099470366)]
[np.float64(0.25010373343311976), np.float64(0.0), np.float64(2.773912405940605), np.float64(0.6167178842009423), np.float64(0.4632339101514613)]


In [ ]:
lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup = toolkits.calc_lower_and_upper_bound_percentile(results_bootstrap_mrnn_obs3_classification_subgroup)

print(lower_bounds_mrnn_obs3_classification_subgroup)
print(upper_bounds_mrnn_obs3_classification_subgroup)

#### Mean values of lower bound and upper bound

In [ ]:
mean_values_ci_mrnn_obs3_classification_subgroup  = toolkits.calc_mean_values_ci(lower_bounds_mrnn_obs3_classification_subgroup, upper_bounds_mrnn_obs3_classification_subgroup)

print(mean_values_ci_mrnn_obs3_classification_subgroup)

[np.float64(0.15155417430747806), np.float64(0.0), np.float64(1.1593962687753612), np.float64(0.2442421185109458), np.float64(0.2088600250259955)]


# Results mean absolute error

In [17]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

#### SAITS

<h4>Standard Scaler (C/Normalização)</h4>

In [13]:
print("SAITS - MAE")
print("************")
toolkits.show_mae(testing_mae_saits_variables_standard, subgroups, variables)

SAITS - MAE
************
General
-------------
ALP : 0.35549184664706085
ALT : 0.34616397235246166
AST : 0.283203929613111
Albumin : 0.5865378684716107
BUN : 0.23549456829210566
Bilirubin : 0.29213390018033225
Cholesterol : 0.8619142944054452
Creatinine : 0.2272360108358054
DiasABP : 0.25696352027850733
FiO2 : 0.4843457493583031
GCS : 0.27782594851598286
Glucose : 0.5574566141755575
HCO3 : 0.42235495223366226
HCT : 0.5478476366595461
HR : 0.2786515469045242
K : 0.6173738181988982
Lactate : 0.3602081488093162
MAP : 0.276148040474521
MechVent : 0.016131509283529688
Mg : 0.5428868319523208
NIDiasABP : 0.23805346595387422
NIMAP : 0.17731126967659114
NISysABP : 0.3059993623684565
Na : 0.5094466185029012
PaCO2 : 0.5448751678244528
PaO2 : 0.5044175503727509
Platelets : 0.37910753219049526
RespRate : 0.4975549864022565
SaO2 : 0.395203639666887
SysABP : 0.2876315375696396
Temp : 0.3654739421516718
TroponinI : 0.5742820858879293
TroponinT : 0.562502744207675
Urine : 0.4555801478350003
WBC : 0.15

#### BRITS

<h4>Standard Scaler (C/Normalização)</h4>

In [47]:
print("BRITS - MAE")
print("************")
toolkits.show_mae(testing_mae_brits_variables_standard, subgroups, variables)

BRITS - MAE
************
General
-------------
ALP : 0.4591248631105258
ALT : 0.2098298479270722
AST : 0.27798859929904113
Albumin : 0.6778925772529109
BUN : 0.39521960833661646
Bilirubin : 0.3833806392125974
Cholesterol : 0.9463970561439892
Creatinine : 0.3444718062995419
DiasABP : 0.20806233248448402
FiO2 : 0.34404522156046397
GCS : 0.21322463397612945
Glucose : 0.5644091100017328
HCO3 : 0.5047606820899914
HCT : 0.6338702223060172
HR : 0.2431173715923441
K : 0.6534995938478387
Lactate : 0.4167839664888235
MAP : 0.1564062923159312
MechVent : 0.007398844359593802
Mg : 0.5955810654854342
NIDiasABP : 0.15725503986364742
NIMAP : 0.09992748699932849
NISysABP : 0.2543230216923866
Na : 0.645442078485974
PaCO2 : 1.1895743136362507
PaO2 : 0.4776378612780617
Platelets : 0.6445916196510731
RespRate : 0.475295251812804
SaO2 : 0.348066606252948
SysABP : 0.25398352012962805
Temp : 0.240561647460938
TroponinI : 0.667214116433886
TroponinT : 0.29949287221387894
Urine : 0.3333883323992429
WBC : 0.1101